# import 

In [1]:
import pandas as pd
import numpy as np
import os
import sys

from astropy.table import Table
from astropy.io import fits
from astropy import units as u
from astropy import constants 
from astropy.coordinates import SkyCoord
from astropy.visualization import hist
from astroML.datasets import fetch_imaging_sample, fetch_sdss_S82standards
from astroML.crossmatch import crossmatch_angular

In [2]:
from scipy.stats.stats import pearsonr
from scipy.stats import spearmanr#
from scipy import stats 

import matplotlib.pyplot as plt
from pylab import cm
import matplotlib as mpl

from collections import OrderedDict
from adjustText import adjust_text

%matplotlib inline
%config InlineBackend.figure_format='svg'
# Edit the font, font size, and axes width
mpl.rcParams['font.family'] = 'Avenir'
plt.rcParams['font.size'] = 18
plt.rcParams['axes.linewidth'] = 2

In [3]:
def drop_index(data):
    data=data.reset_index(drop=True)
    return data

In [4]:
from astropy.cosmology import FlatLambdaCDM,Planck13,Planck15,z_at_value
from astropy import units as u
from astropy.cosmology import LambdaCDM
cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7)
#cosmo=FlatLambdaCDM(H0=73,Om0=0.3)

In [5]:
import numpy as np
from astropy import constants
from astropy import units as u
from astropy import units

wave6=np.array([5468,4392,3465,2600,2246,1928])# v,b,u,w1,m2,w2
freq6=(constants.c/(wave6*units.AA)).to(u.Hz).value

# datadir

In [6]:
CLAGN_statistic_plot_dir='/Volumes/ExtremeSSD/CLAGN_statistic_plot/'

CLAGN_wisedata_dir='/Users/brettlv/blog/brettlv.github.io/pythoncode/changinglookAGN/CLAGN_wise_data/'

AGN_wisedata_dir='/Volumes/ExtremeSSD/data_source/WISE_data'
LLAGN_wisedata_dir='%s/wise_data_search/LLAGN'%AGN_wisedata_dir
QSO_wisedata_dir='%s/wise_data_search/QSO'%AGN_wisedata_dir


AGN_MAXIdata_dir='/Volumes/ExtremeSSD/CLAGN_statistic_plot/MAXIlc/MAXI/'#%s_MAXI_1day_all.dat'%name

ls /Volumes/ExtremeSSD/CLAGN_statistic_plot/MAXIlc/MAXI/

## set_ax

In [7]:
#from adjustText import adjust_text

def set_ax_tick(ax):
    ax.xaxis.set_tick_params(which='major', size=10, width=2, direction='in', top='on',)
    ax.xaxis.set_tick_params(which='minor', size=5, width=2, direction='in', top='on')
    ax.yaxis.set_tick_params(which='major', size=10, width=2, direction='in', right='on')
    ax.yaxis.set_tick_params(which='minor', size=5, width=2, direction='in', right='on')

def set_ax_locator(ax,xma=1,xmi=0.2,yma=1,ymi=0.2):
    ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(xma))
    ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(xmi))
    ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(yma))
    ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(ymi)) 

## timeconvert

In [8]:
from astropy.time import Time
from astropy.io import fits
import time
import datetime
from matplotlib.pyplot import MultipleLocator
import matplotlib.dates as mdates

def datetime2mjd(x):
    mjd_ref=59000
    mjd_minus_mdates_num=mdates.date2num(convert_xaxis_time(mjd_ref))-mjd_ref
    
    x=mdates.date2num(x)
    y = x - mjd_minus_mdates_num   
    return y

def mjd2datetime(x):
    mjd_ref=59000
    mjd_minus_mdates_num=mdates.date2num(convert_xaxis_time(mjd_ref))-mjd_ref
    y= x + mjd_minus_mdates_num
    y= mdates.num2date(y)
    return y



def datenums2mjd(x):
    #x=mdates.date2num(x)
    mjd_ref=59000
    mjd_minus_mdates_num=mdates.date2num(convert_xaxis_time(mjd_ref))-mjd_ref
    y = x - mjd_minus_mdates_num   
    return y

def mjd2numsdate(x):
    mjd_ref=59000
    mjd_minus_mdates_num=mdates.date2num(convert_xaxis_time(mjd_ref))-mjd_ref
    
    y= x + mjd_minus_mdates_num
    #y= mdates.num2date(y)
    return y


def convert_xaxis_mjd(time):
    return Time(time).mjd   

def convert_xaxis_time(mjd):
    return Time(mjd,format='mjd').to_datetime()


def date2yday(x):
    """
    x is in matplotlib datenums, so they are floats.
    """
    y = x - mdates.date2num(datetime(2018, 1, 1))
    return y

def yday2date(x):
    """
    return a matplotlib datenum (x is days since start of year of 2018)
    """
    y = x + mdates.date2num(datetime(2018, 1, 1))
    return y


def convert_partial_year(numbers):
    datetimes=[]
    for number in numbers:
        year = int(number)
        d = timedelta(days=(number - year)*(365 + is_leap(year)))
        day_one = datetime(year,1,1)
        date = d + day_one
        datetimes.append(date)
    return datetimes


def is_leap(year):
    if not year%4 and  year%100 or not year%400:
        return True
    return False


def convert_mjd(times):
    timesmjd=[]
    for i in times:
        timesmjd.append(Time(i).mjd)
    return timesmjd  


def convert_date(times):
    timesdate=[]
    for i in times:
        timesdate.append(Time(i,format='mjd').datetime)
    return timesdate

def convert_date_single(time):
    timedate=Time(time,format='mjd').datetime
    return timedate

In [9]:
def plot_secax(ax,mi_interval=365,ma_interval=365*2):
    secax1 = ax.secondary_xaxis('top', functions=(mjd2numsdate,datenums2mjd))
    secax1.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
    secax1.xaxis.set_major_locator(mdates.DayLocator(interval=ma_interval))
    secax1.xaxis.set_minor_locator(mdates.DayLocator(interval=mi_interval))
    secax1.xaxis.set_tick_params(which='major', size=10, width=2, direction='out')
    secax1.xaxis.set_tick_params(which='minor', size=5, width=2, direction='out')
    secax1.xaxis.set_tick_params(which='major', size=10, width=2, direction='out', rotation=45,)

mjds=np.linspace(59000,59120,30)
fluxs=np.sin(np.arange(len(mjds)))+np.random.random(len(mjds))*0.2

fig = plt.figure(figsize=(4,4))
fig.subplots_adjust(hspace=0.15, wspace = 0.3)
ax = fig.add_subplot(111)

ax.scatter(mjds,fluxs)
plot_secax(ax,90,30)
set_ax_tick(ax)
set_ax_locator(ax,xma=60,xmi=30,yma=1.0,ymi=0.5)
ax.text(0.03, 0.85, 'test', fontsize = 15, transform = ax.transAxes)
plt.show()
#fig.savefig('/Users/brettlv/Downloads/CLAGN_statistic_plot/WISE_lc/rebin_CLAGN/rebin_gt50_gt0.5mag//W1_rebinlc_%s.png'%save_name,dpi=400,transparent=False,bbox_inches='tight')      
plt.close()

In [ ]:
convert_xaxis_time(59100)

In [ ]:
convert_xaxis_mjd(datetime.datetime(2020, 9, 8, 0, 0))

## get_new_range_datas
(mjd1, flux1, err1,time_min,time_max)

In [10]:
def get_new_range_datas(mjd1, flux1, err1,time_min,time_max):
    mjd_s=[]
    flux_s=[]
    err_s=[]
    for i,j,k in zip(mjd1, flux1, err1):
        if i>=time_min and i<time_max:
            mjd_s.append(i)
            flux_s.append(j)
            err_s.append(k)
    return np.array(mjd_s),np.array(flux_s),np.array(err_s)

## AB mag

m(AB) = -2.5 log(f) - 48.60

# rebin_wiselc

## mag_weighted_mean_mag

In [11]:
import numpy as np
def get_mag_weighted_mean_mag(mags,e_mags):
    N=len(mags)
    mag_weighted_mean_mag =0  
    
    w_n=np.sum(1.0/e_mags**2)   
    
    for mag,e_mag in zip(mags,e_mags):
        w_i=1.0/e_mag**2
        w=w_i/w_n
        mag_weighted_mean_mag=mag_weighted_mean_mag+w*mag
                          
    return mag_weighted_mean_mag 


def get_mag_weighted_mean_e_mag(mags,e_mags):
    N=len(mags)  

    w_n=np.sum(1.0/e_mags**2)
    
    mag_weighted_mean_mag = get_mag_weighted_mean_mag(mags,e_mags)
    
    mag_weighted_mean_e_mag =0 
    
    for mag,e_mag in zip(mags,e_mags):        
        w_i=1.0/e_mag**2
        w=w_i/w_n
        
        mag_weighted_mean_e_mag=mag_weighted_mean_e_mag+w*(mag-mag_weighted_mean_mag)**2
        
    mag_weighted_mean_e_mag_=(mag_weighted_mean_e_mag/(N-1))**0.5                                 
    return mag_weighted_mean_e_mag_

In [ ]:
mags=np.array([7,7.2,7.1,7.2,6.9,7.1,10.2])
e_mags=np.array([0.02,0.04,0.05,0.03,0.02,0.6])
mean_mag=get_mag_weighted_mean_mag(mags,e_mags)
mean_e_mag=get_mag_weighted_mean_e_mag(mags,e_mags)
print(np.mean(mags),mean_mag)
print(np.mean(e_mags),mean_e_mag)

# median_lc_without_outlier

## def_median_lc

In [12]:
def median_lc(data,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=10):
    data=data.sort_values(timelabel)
    data=drop_index(data)
    mean_mjds=[]
    mean_mags=[]
    mean_mags_e=[]
    if len(data)>2:
        mjd_t0=data[timelabel][0]-20
        mjd_tend=data[timelabel][len(data)-1]+20

        N=int((mjd_tend-mjd_t0)//interval+1)
        #print(N)

        time_interval=np.linspace(mjd_t0,interval*N+mjd_t0,N+1)
        #print(time_interval)

        for index,i in enumerate(time_interval):
            if index < N:
                data_select=data[data[timelabel]>=time_interval[index]]
                data_select=data_select[data_select[timelabel]<time_interval[index+1]]
                data_select=drop_index(data_select)

                if len(data_select)>2:
                    #mean_mjd=np.mean(data_select[timelabel])
                    mags=data_select[maglabel]
                    e_mags=data_select[e_maglabel]               
                    mean_mag=get_mag_weighted_mean_mag(mags,e_mags)
                    mag_weighted_mean_e_mag=get_mag_weighted_mean_e_mag(mags,e_mags)
                    #mean_mag_e=np.mean(data_select[e_maglabel])

                    data_select=data_select[abs(data_select[maglabel]-mean_mag)<sigma_outlier*mag_weighted_mean_e_mag]
                    data_select=drop_index(data_select)


                    #print(len(data_select))

                    mags=data_select[maglabel]
                    e_mags=data_select[e_maglabel]  
                    if len(data_select)>2:
                        mean_mag=get_mag_weighted_mean_mag(mags,e_mags)               
                        mean_mag_e=get_mag_weighted_mean_e_mag(mags,e_mags)

                        mean_mjd=np.mean(data_select[timelabel])               
                        mean_mjds.append(mean_mjd)
                        mean_mags.append(mean_mag)
                        mean_mags_e.append(mean_mag_e)
        mean_mjds,mean_mags,mean_mags_e=(np.array(mean_mjds),np.array(mean_mags),np.array(mean_mags_e))
    return mean_mjds,mean_mags,mean_mags_e


## plot_outlier_median_lc

In [13]:
def plot_outlier_median_lc(ax,data,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=10):
    data=data.sort_values(timelabel)
    data=drop_index(data)
    mean_mjds=[]
    mean_mags=[]
    mean_mags_e=[]
    if len(data)>2:
        mjd_t0=data[timelabel][0]-20
        mjd_tend=data[timelabel][len(data)-1]+20

        N=int((mjd_tend-mjd_t0)//interval+1)
        #print(N)

        time_interval=np.linspace(mjd_t0,interval*N+mjd_t0,N+1)
        #print(time_interval)

        for index,i in enumerate(time_interval):
            if index < N:
                data_select=data[data[timelabel]>=time_interval[index]]
                data_select=data_select[data_select[timelabel]<time_interval[index+1]]
                data_select=drop_index(data_select)

                if len(data_select)>2:
                    #mean_mjd=np.mean(data_select[timelabel])
                    mags=data_select[maglabel]
                    e_mags=data_select[e_maglabel]               
                    mean_mag=get_mag_weighted_mean_mag(mags,e_mags)
                    mag_weighted_mean_e_mag=get_mag_weighted_mean_e_mag(mags,e_mags)
                    #mean_mag_e=np.mean(data_select[e_maglabel])

                    data_select_outlier=data_select[abs(data_select[maglabel]-mean_mag)>=sigma_outlier*mag_weighted_mean_e_mag]
                    data_select_outlier=drop_index(data_select_outlier)  
                    if len(data_select_outlier)>0:
                        ax.scatter(data_select_outlier[timelabel],data_select_outlier[maglabel],
                               color='red',zorder=3,s=6)


## def_plot_outliers

In [14]:
def plot_outliers_w1(ax,data_test_neo,data_test_mep,sigma_outlier=10):
    timelabel,maglabel,e_maglabel=('mjd','w1mpro','w1sigmpro')
    plot_outlier_median_lc(ax,data_test_neo,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=sigma_outlier)

    timelabel,maglabel,e_maglabel=('mjd','w1mpro_ep','w1sigmpro_ep')
    plot_outlier_median_lc(ax,data_test_mep,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=sigma_outlier)

    #return data_test_neo_w1_outlier,data_test_neo_w2_outlier,data_test_mep_w1_outlier,data_test_mep_w2_outlier

def plot_outliers_w2(ax,data_test_neo,data_test_mep,sigma_outlier=10):
    timelabel,maglabel,e_maglabel=('mjd','w2mpro','w2sigmpro')
    plot_outlier_median_lc(ax,data_test_neo,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=sigma_outlier)

    timelabel,maglabel,e_maglabel=('mjd','w2mpro_ep','w2sigmpro_ep')
    plot_outlier_median_lc(ax,data_test_mep,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=sigma_outlier)

# getdata

## get_wise_data

In [15]:
def get_wise_data(name_test,datawise_dir=CLAGN_wisedata_dir):
    #print(name_test)
    search_name=name_test.replace(' ','') 
    save_name=search_name.strip()

    data_test_neo=pd.read_csv(os.path.join(datawise_dir,'wise_%s_neo.csv'%save_name))
    data_test_neo=data_test_neo[data_test_neo['w1mpro'].notnull()]
    data_test_neo=data_test_neo[data_test_neo['w2mpro'].notnull()]
    data_test_neo=data_test_neo[data_test_neo['w1sigmpro'].notnull()]
    data_test_neo=data_test_neo[data_test_neo['w2sigmpro'].notnull()]
    data_test_neo=data_test_neo[data_test_neo['w1mpro']<15]
    data_test_neo=data_test_neo[data_test_neo['w2mpro']<13]  
    #data_test_neo=data_test_neo[data_test_neo['w1rchi2']<2]
    #data_test_neo=data_test_neo[data_test_neo['w2rchi2']<2]
    data_test_neo=data_test_neo[data_test_neo['cc_flags']=="b'0000'"]
    data_test_neo=data_test_neo[data_test_neo['nb']<3]
    data_test_neo=data_test_neo[data_test_neo['na']==0] 
    
    data_test_neo=data_test_neo[data_test_neo['qual_frame']>0]
    #print(len(data_test_neo))
    data_test_neo=(data_test_neo.loc[:, ['mjd', 'w1mpro','w2mpro',
                                 'w1sigmpro','w2sigmpro',
                                 ]].sort_values('mjd'))

    data_test_neo=data_test_neo.reset_index(drop=True)
    #print(data_test_neo.columns)

    data_test_mep=pd.read_csv(os.path.join(datawise_dir,'wise_%s_mep.csv'%save_name))
    data_test_mep=data_test_mep[data_test_mep['w1mpro_ep'].notnull()]
    data_test_mep=data_test_mep[data_test_mep['w2mpro_ep'].notnull()]
    data_test_mep=data_test_mep[data_test_mep['w1sigmpro_ep'].notnull()]
    data_test_mep=data_test_mep[data_test_mep['w2sigmpro_ep'].notnull()]
    #data_test_mep=data_test_mep[data_test_mep['qual_frame']>5]
    data_test_mep=data_test_mep[data_test_mep['qi_fact']>0]
    data_test_mep=data_test_mep[data_test_mep['saa_sep']>0]
    data_test_mep=data_test_mep[data_test_mep['cc_flags']=="b'0000'"]
    data_test_mep=data_test_mep[data_test_mep['nb']<3]
    data_test_mep=data_test_mep[data_test_mep['na']==0] 

    #print(len(data_test_mep))
    data_test_mep=(data_test_mep.loc[:, ['mjd', 'w1mpro_ep','w2mpro_ep',
                                         'w1sigmpro_ep','w2sigmpro_ep',
                                         ]].sort_values('mjd'))

    data_test_mep=data_test_mep.reset_index(drop=True)
    #print(data_test_mep.columns)
    #if not os.path.exists(save_lc_img_path):
      
    return data_test_neo,data_test_mep
    

## def get_intrinsic_var

In [16]:
epsilon_s_w1=0.024
epsilon_s_w2=0.028

def get_intrinsic_var(data,mag_label,magerr_label,epsilon_s):
    
    mags=data[mag_label]
    e_mags=data[magerr_label]
    
    mag_weighted_mean_mag = get_mag_weighted_mean_mag(mags,e_mags)    
    mag_weighted_mean_e_mag_= get_mag_weighted_mean_e_mag(mags,e_mags)
    mag_mean=mag_weighted_mean_mag
    
    length_N=len(data[mag_label])
    epsilon_square=np.average(data[magerr_label]**2)+epsilon_s**2
    Sigma_square=np.sum((data[mag_label]-mag_mean)**2)/(length_N-1)
    
    if Sigma_square>epsilon_square:
        sigma_m=np.sqrt(Sigma_square-epsilon_square)
    else:
        sigma_m=0
    
    if length_N<2:
        sigma_m=-1#print(length_N)
    return sigma_m

### CLAGN

In [17]:
CLAGN_210919=pd.read_excel('/Users/brettlv/blog/IHEP_sync/data/data_clagn/CLAGN_list210919.xlsx',sheet_name='Sheet1')

In [18]:
epsilon_s_w1=0.024
epsilon_s_w2=0.028

for index,name_test in enumerate(CLAGN_210919['Name']): 
    #print(name_test)
    search_name=name_test.replace(' ','') 
    save_name=search_name.strip()
    data_test_neo,data_test_mep= get_wise_data(name_test,datawise_dir='/Users/brettlv/blog/brettlv.github.io/pythoncode/changinglookAGN/CLAGN_wise_data/')
    
    if len(data_test_neo)>2:
        mag_label='w1mpro'
        magerr_label='w1sigmpro'
        epsilon_s=epsilon_s_w1

        pearson_coeff,p_value=pearsonr(data_test_neo['w1mpro'],data_test_neo['w2mpro'])
        CLAGN_210919.loc[index,'pearson_coeff_neo']=pearson_coeff

        sigma_m_w1=get_intrinsic_var(data_test_neo,mag_label,magerr_label,epsilon_s)
        CLAGN_210919.loc[index,'sigma_m_w1_neo']=sigma_m_w1
        
        mags=data_test_neo[mag_label]
        e_mags=data_test_neo[magerr_label]
        
        CLAGN_210919.loc[index,'mean_m_w1_neo']=get_mag_weighted_mean_mag(mags,e_mags)
        CLAGN_210919.loc[index,'mean_e_m_w1_neo']=get_mag_weighted_mean_e_mag(mags,e_mags)


        mag_label='w2mpro'
        magerr_label='w2sigmpro'
        epsilon_s=epsilon_s_w2
        sigma_m_w2=get_intrinsic_var(data_test_neo,mag_label,magerr_label,epsilon_s)
        CLAGN_210919.loc[index,'sigma_m_w2_neo']=sigma_m_w2
        
        mags=data_test_neo[mag_label]
        e_mags=data_test_neo[magerr_label]
        
        CLAGN_210919.loc[index,'mean_m_w2_neo']=get_mag_weighted_mean_mag(mags,e_mags)
        CLAGN_210919.loc[index,'mean_e_m_w2_neo']=get_mag_weighted_mean_e_mag(mags,e_mags)
        
        CLAGN_210919.loc[index,'len_neo']=len(data_test_neo)
        CLAGN_210919.loc[index,'W1-W2']=np.average(data_test_neo['w1mpro']-data_test_neo['w2mpro'])


    else:
        CLAGN_210919.loc[index,'len_neo']=0  

In [19]:
len(CLAGN_210919[CLAGN_210919['len_neo']>2]),len(CLAGN_210919[CLAGN_210919['len_neo']>20])

(93, 86)

### CLAGN_210919_logLw

In [20]:
CLAGN_210919['D_cm']=cosmo.luminosity_distance(z=CLAGN_210919['Redshift']).to(u.cm)
CLAGN_210919['logL_W1']= (-48.6-CLAGN_210919['mean_m_w1_neo']-2.699)/2.5+np.log10(4*np.pi*CLAGN_210919['D_cm']**2)+np.log10((constants.c/(3.4*u.um)).to(u.Hz).value)
CLAGN_210919['logL_W2']= (-48.6-CLAGN_210919['mean_m_w2_neo']-3.339)/2.5+np.log10(4*np.pi*CLAGN_210919['D_cm']**2)+np.log10((constants.c/(4.6*u.um)).to(u.Hz).value)

CLAGN_210919['e_logL_W1'] =   CLAGN_210919['mean_e_m_w1_neo']/2.5
CLAGN_210919['e_logL_W2'] =   CLAGN_210919['mean_e_m_w2_neo']/2.5
    

In [21]:
CLAGN_210919_mass=CLAGN_210919[CLAGN_210919['lg(MBH/Msun)'].notnull()]
CLAGN_210919_mass=CLAGN_210919_mass.reset_index(drop=True)
CLAGN_210919_mass['logL_W1/Ledd']=CLAGN_210919_mass['logL_W1']-CLAGN_210919_mass['lg(MBH/Msun)']-np.log10(1.263e38)
CLAGN_210919_mass['logL_W2/Ledd']=CLAGN_210919_mass['logL_W2']-CLAGN_210919_mass['lg(MBH/Msun)']-np.log10(1.263e38)

In [22]:
CLAGN_210919_mass.columns

Index(['Name', 'RA', 'DEC', 'Ref.', 'Type_Ref', 'Redshift', 'lg(MBH/Msun)',
       'Ref..1', 'mass_Ref', 'Type_CL', 'Type_X', 'OX', 'logM', 'pos',
       'E(B-V)mean', 'E(B-V)weighted', 'NHImean', 'NHIweighted', 'NH2mean',
       'NH2weighted', 'NH,totmean', 'NH,totweighted', 'len_neo',
       'pearson_coeff_neo', 'sigma_m_w1_neo', 'mean_m_w1_neo',
       'mean_e_m_w1_neo', 'sigma_m_w2_neo', 'mean_m_w2_neo', 'mean_e_m_w2_neo',
       'W1-W2', 'D_cm', 'logL_W1', 'logL_W2', 'e_logL_W1', 'e_logL_W2',
       'logL_W1/Ledd', 'logL_W2/Ledd'],
      dtype='object')

In [ ]:
np.mean(CLAGN_210919_mass['logL_W1']-CLAGN_210919_mass['lg(MBH/Msun)']-np.log10(1.263*10**38))

In [ ]:
10**-2.317

CLAGN_210919_mass.to_csv('%s/CLAGN_210919_mass_wisevar.csv'%CLAGN_statistic_plot_dir,index=False)

In [ ]:
len(CLAGN_210919),len(CLAGN_210919_mass)

In [ ]:
len(CLAGN_210919_mass[CLAGN_210919_mass['len_neo']>2]),len(CLAGN_210919_mass[CLAGN_210919_mass['len_neo']>20])

### mass \sigma>0.1

In [ ]:
sigmadot1=CLAGN_210919_mass[CLAGN_210919_mass['sigma_m_w1_neo']>0.1][['Name','sigma_m_w1_neo','OX']]

In [ ]:
len(sigmadot1)

In [ ]:
sigmadot1_X=sigmadot1[sigmadot1['OX']==' B ']
sigmadot1_O=sigmadot1[sigmadot1['OX']==' A ']

### MAXI_AGN_namelist

In [ ]:
MAXI_AGN_namelist=[]
files=os.listdir(AGN_MAXIdata_dir)
for file in files:
    if file.endswith('.dat'):
        #print(file[:-18])
        if file[:-18] not in MAXI_AGN_namelist:
            MAXI_AGN_namelist.append(file[:-18])

In [ ]:
for i in MAXI_AGN_namelist:
    if i in CLAGN_210919['Name'].values:
        print(i)

### LLAGN_gu2009

In [23]:
def get_LLAGNwise_data(name_test,datawise_dir):
    #print(name_test)
    search_name=name_test.replace(' ','') 
    save_name=name_test

    data_test_neo=pd.read_csv(os.path.join(datawise_dir,'wise_%s_neo.csv'%save_name))
    data_test_neo=data_test_neo[data_test_neo['w1mpro'].notnull()]
    data_test_neo=data_test_neo[data_test_neo['w2mpro'].notnull()]
    data_test_neo=data_test_neo[data_test_neo['w1sigmpro'].notnull()]
    data_test_neo=data_test_neo[data_test_neo['w2sigmpro'].notnull()]
    data_test_neo=data_test_neo[data_test_neo['w1mpro']<15]
    data_test_neo=data_test_neo[data_test_neo['w2mpro']<13]  
    #data_test_neo=data_test_neo[data_test_neo['w1rchi2']<2]
    #data_test_neo=data_test_neo[data_test_neo['w2rchi2']<2]
    data_test_neo=data_test_neo[data_test_neo['cc_flags']=="b'0000'"]
    data_test_neo=data_test_neo[data_test_neo['nb']<3]
    data_test_neo=data_test_neo[data_test_neo['na']==0] 
    
    data_test_neo=data_test_neo[data_test_neo['qual_frame']>0]
    #print(len(data_test_neo))
    data_test_neo=(data_test_neo.loc[:, ['mjd', 'w1mpro','w2mpro',
                                 'w1sigmpro','w2sigmpro',
                                 ]].sort_values('mjd'))

    data_test_neo=data_test_neo.reset_index(drop=True)
    #print(data_test_neo.columns)

    data_test_mep=pd.read_csv(os.path.join(datawise_dir,'wise_%s_mep.csv'%save_name))
    data_test_mep=data_test_mep[data_test_mep['w1mpro_ep'].notnull()]
    data_test_mep=data_test_mep[data_test_mep['w2mpro_ep'].notnull()]
    data_test_mep=data_test_mep[data_test_mep['w1sigmpro_ep'].notnull()]
    data_test_mep=data_test_mep[data_test_mep['w2sigmpro_ep'].notnull()]
    #data_test_mep=data_test_mep[data_test_mep['qual_frame']>5]
    data_test_mep=data_test_mep[data_test_mep['qi_fact']>0]
    data_test_mep=data_test_mep[data_test_mep['saa_sep']>0]
    data_test_mep=data_test_mep[data_test_mep['cc_flags']=="b'0000'"]
    data_test_mep=data_test_mep[data_test_mep['nb']<3]
    data_test_mep=data_test_mep[data_test_mep['na']==0] 

    #print(len(data_test_mep))
    data_test_mep=(data_test_mep.loc[:, ['mjd', 'w1mpro_ep','w2mpro_ep',
                                         'w1sigmpro_ep','w2sigmpro_ep',
                                         ]].sort_values('mjd'))

    data_test_mep=data_test_mep.reset_index(drop=True)
    #print(data_test_mep.columns)
    #if not os.path.exists(save_lc_img_path):
      
    return data_test_neo,data_test_mep
    

In [24]:
LLAGN_gu2009=pd.read_excel('/Users/brettlv/blog/IHEP_sync/data/AGNmass/LLAGN_QSO/LLAGN_Gu2009.xlsx',sheet_name='Sheet1')
LLAGN_gu2009=LLAGN_gu2009[LLAGN_gu2009['CLAGN?'].isnull()]
LLAGN_gu2009=drop_index(LLAGN_gu2009)
print(len(LLAGN_gu2009))

for index,(name_,z_) in enumerate(zip(LLAGN_gu2009['Name'],LLAGN_gu2009['z'])):
    if name_ =='NGC 3031':
        LLAGN_gu2009.loc[index,'D_cm']=3.5*u.Mpc.to(u.cm)
        #print(name_,3.5*u.Mpc.to(u.cm))
    else:
        #print(name_, '%.6f'%z_)
        D_cm=cosmo.luminosity_distance(z_).to(u.cm).value
        #print(D_cm)
        LLAGN_gu2009.loc[index,'D_cm']=D_cm 

49


In [25]:
LLAGN_gu2009.columns

Index(['Name', 'z', 'Sat', '(2–10 keV)', 'log L2−10keV', 'References',
       '∗ log MBH', 'References.1', 'Type', 'CLAGN?', 'D_cm'],
      dtype='object')

In [ ]:
np.mean(LLAGN_gu2009['log L2−10keV']+np.log10(30)-LLAGN_gu2009['∗ log MBH']-np.log10(1.263*10**38))

LLAGN_gu2009[['Name','D_cm']]

In [ ]:
LLAGN_gu2009.columns

In [26]:
epsilon_s_w1=0.024
epsilon_s_w2=0.028

for index,name_test in enumerate(LLAGN_gu2009['Name']): 
    #print(name_test)
    search_name=name_test.replace(' ','') 
    save_name=name_test
    
    LLAGN_wisedata_dir='%s/wise_data_search/LLAGN'%AGN_wisedata_dir
    data_test_neo,data_test_mep= get_LLAGNwise_data(name_test,datawise_dir=LLAGN_wisedata_dir)
    
    if len(data_test_neo)>2:
        mag_label='w1mpro'
        magerr_label='w1sigmpro'
        epsilon_s=epsilon_s_w1

        pearson_coeff,p_value=pearsonr(data_test_neo['w1mpro'],data_test_neo['w2mpro'])
        LLAGN_gu2009.loc[index,'pearson_coeff_neo']=pearson_coeff

        sigma_m_w1=get_intrinsic_var(data_test_neo,mag_label,magerr_label,epsilon_s)
        LLAGN_gu2009.loc[index,'sigma_m_w1_neo']=sigma_m_w1
        
        mags=data_test_neo[mag_label]
        e_mags=data_test_neo[magerr_label]
        
        LLAGN_gu2009.loc[index,'mean_m_w1_neo']=get_mag_weighted_mean_mag(mags,e_mags)
        LLAGN_gu2009.loc[index,'mean_e_m_w1_neo']=get_mag_weighted_mean_e_mag(mags,e_mags)


        mag_label='w2mpro'
        magerr_label='w2sigmpro'
        epsilon_s=epsilon_s_w2
        sigma_m_w2=get_intrinsic_var(data_test_neo,mag_label,magerr_label,epsilon_s)
        LLAGN_gu2009.loc[index,'sigma_m_w2_neo']=sigma_m_w2
        
        mags=data_test_neo[mag_label]
        e_mags=data_test_neo[magerr_label]
        
        LLAGN_gu2009.loc[index,'mean_m_w2_neo']=get_mag_weighted_mean_mag(mags,e_mags)
        LLAGN_gu2009.loc[index,'mean_e_m_w2_neo']=get_mag_weighted_mean_e_mag(mags,e_mags)
        
        LLAGN_gu2009.loc[index,'len_neo']=len(data_test_neo)
        LLAGN_gu2009.loc[index,'W1-W2']=np.average(data_test_neo['w1mpro']-data_test_neo['w2mpro'])


    else:
        LLAGN_gu2009.loc[index,'len_neo']=0  

In [27]:
LLAGN_gu2009.columns

Index(['Name', 'z', 'Sat', '(2–10 keV)', 'log L2−10keV', 'References',
       '∗ log MBH', 'References.1', 'Type', 'CLAGN?', 'D_cm',
       'pearson_coeff_neo', 'sigma_m_w1_neo', 'mean_m_w1_neo',
       'mean_e_m_w1_neo', 'sigma_m_w2_neo', 'mean_m_w2_neo', 'mean_e_m_w2_neo',
       'len_neo', 'W1-W2'],
      dtype='object')

In [28]:
LLAGN_gu2009['logL_W1']= (-48.6-LLAGN_gu2009['mean_m_w1_neo']-2.699)/2.5+np.log10(4*np.pi*LLAGN_gu2009['D_cm']**2)+np.log10((constants.c/(3.4*u.um)).to(u.Hz).value)
LLAGN_gu2009['logL_W2']= (-48.6-LLAGN_gu2009['mean_m_w2_neo']-3.339)/2.5+np.log10(4*np.pi*LLAGN_gu2009['D_cm']**2)+np.log10((constants.c/(4.6*u.um)).to(u.Hz).value)


LLAGN_gu2009['e_logL_W1'] =   LLAGN_gu2009['mean_e_m_w1_neo']/2.5
LLAGN_gu2009['e_logL_W2'] =   LLAGN_gu2009['mean_e_m_w2_neo']/2.5
    
    
LLAGN_gu2009['logL_W1/Ledd']=LLAGN_gu2009['logL_W1']-LLAGN_gu2009['∗ log MBH']-np.log10(1.263e38)
LLAGN_gu2009['logL_W2/Ledd']=LLAGN_gu2009['logL_W2']-LLAGN_gu2009['∗ log MBH']-np.log10(1.263e38)


In [29]:
np.mean(LLAGN_gu2009['logL_W1']-LLAGN_gu2009['∗ log MBH']-np.log10(1.263*10**38))

-3.454805332934524

In [ ]:
10**-3.454

In [ ]:
CLAGN_statistic_plot_dir

In [ ]:
LLAGN_gu2009.columns

In [ ]:
LLAGN_gu2009.to_csv('%s/LLAGN_gu2009.csv'%CLAGN_statistic_plot_dir,index=False)

### QSO

In [30]:
QSO=pd.read_csv('%s/QSOtable.txt'%AGN_wisedata_dir,delimiter='|',header=None)
QSO.columns=['Name', 'redshift', 'FWHM_Halpha', 'lg(LHalpha)', 'lg(L_Mbh)']#
#Irsa.ROW_LIMIT = 1000 # value of new row limit here.
#Irsa.TIMEOUT = 120

In [31]:
def get_QSOwise_data(name_test,datawise_dir):
    #print(name_test)
    search_name='SDSS'+name_test.replace(' ','')
    save_name='SDSS '+name_test.replace(' ','')

    data_test_neo=pd.read_csv(os.path.join(datawise_dir,'wise_%s_neo.csv'%save_name))
    data_test_neo=data_test_neo[data_test_neo['w1mpro'].notnull()]
    data_test_neo=data_test_neo[data_test_neo['w2mpro'].notnull()]
    data_test_neo=data_test_neo[data_test_neo['w1sigmpro'].notnull()]
    data_test_neo=data_test_neo[data_test_neo['w2sigmpro'].notnull()]
    data_test_neo=data_test_neo[data_test_neo['w1mpro']<15]
    data_test_neo=data_test_neo[data_test_neo['w2mpro']<13]  
    #data_test_neo=data_test_neo[data_test_neo['w1rchi2']<2]
    #data_test_neo=data_test_neo[data_test_neo['w2rchi2']<2]
    data_test_neo=data_test_neo[data_test_neo['cc_flags']=="b'0000'"]
    data_test_neo=data_test_neo[data_test_neo['nb']<3]
    data_test_neo=data_test_neo[data_test_neo['na']==0] 
    
    data_test_neo=data_test_neo[data_test_neo['qual_frame']>0]
    #print(len(data_test_neo))
    data_test_neo=(data_test_neo.loc[:, ['mjd', 'w1mpro','w2mpro',
                                 'w1sigmpro','w2sigmpro',
                                 ]].sort_values('mjd'))

    data_test_neo=data_test_neo.reset_index(drop=True)
    #print(data_test_neo.columns)

    data_test_mep=pd.read_csv(os.path.join(datawise_dir,'wise_%s_mep.csv'%save_name))
    data_test_mep=data_test_mep[data_test_mep['w1mpro_ep'].notnull()]
    data_test_mep=data_test_mep[data_test_mep['w2mpro_ep'].notnull()]
    data_test_mep=data_test_mep[data_test_mep['w1sigmpro_ep'].notnull()]
    data_test_mep=data_test_mep[data_test_mep['w2sigmpro_ep'].notnull()]
    #data_test_mep=data_test_mep[data_test_mep['qual_frame']>5]
    data_test_mep=data_test_mep[data_test_mep['qi_fact']>0]
    data_test_mep=data_test_mep[data_test_mep['saa_sep']>0]
    data_test_mep=data_test_mep[data_test_mep['cc_flags']=="b'0000'"]
    data_test_mep=data_test_mep[data_test_mep['nb']<3]
    data_test_mep=data_test_mep[data_test_mep['na']==0] 

    #print(len(data_test_mep))
    data_test_mep=(data_test_mep.loc[:, ['mjd', 'w1mpro_ep','w2mpro_ep',
                                         'w1sigmpro_ep','w2sigmpro_ep',
                                         ]].sort_values('mjd'))

    data_test_mep=data_test_mep.reset_index(drop=True)
    #print(data_test_mep.columns)
    #if not os.path.exists(save_lc_img_path):
      
    return data_test_neo,data_test_mep
    

In [32]:
epsilon_s_w1=0.024
epsilon_s_w2=0.028

for index,name in enumerate(QSO['Name']):
    #print(i)
    search_name='SDSS'+name.replace(' ','')
    save_name='SDSS '+name.replace(' ','') 
    
    QSO_wisedata_dir='%s/wise_data_search/QSO'%AGN_wisedata_dir
    data_test_neo,data_test_mep= get_QSOwise_data(name,datawise_dir=QSO_wisedata_dir)
    
    if len(data_test_neo)>2:
        mag_label='w1mpro'
        magerr_label='w1sigmpro'
        epsilon_s=epsilon_s_w1

        pearson_coeff,p_value=pearsonr(data_test_neo['w1mpro'],data_test_neo['w2mpro'])
        QSO.loc[index,'pearson_coeff_neo']=pearson_coeff

        sigma_m_w1=get_intrinsic_var(data_test_neo,mag_label,magerr_label,epsilon_s)
        QSO.loc[index,'sigma_m_w1_neo']=sigma_m_w1
        
        mags=data_test_neo[mag_label]
        e_mags=data_test_neo[magerr_label]
        
        QSO.loc[index,'mean_m_w1_neo']=get_mag_weighted_mean_mag(mags,e_mags)
        QSO.loc[index,'mean_e_m_w1_neo']=get_mag_weighted_mean_e_mag(mags,e_mags)


        mag_label='w2mpro'
        magerr_label='w2sigmpro'
        epsilon_s=epsilon_s_w2
        sigma_m_w2=get_intrinsic_var(data_test_neo,mag_label,magerr_label,epsilon_s)
        QSO.loc[index,'sigma_m_w2_neo']=sigma_m_w2
        
        mags=data_test_neo[mag_label]
        e_mags=data_test_neo[magerr_label]
        
        QSO.loc[index,'mean_m_w2_neo']=get_mag_weighted_mean_mag(mags,e_mags)
        QSO.loc[index,'mean_e_m_w2_neo']=get_mag_weighted_mean_e_mag(mags,e_mags)
        
        QSO.loc[index,'len_neo']=len(data_test_neo)
        QSO.loc[index,'W1-W2']=np.average(data_test_neo['w1mpro']-data_test_neo['w2mpro'])


    else:
        QSO.loc[index,'len_neo']=0  

In [33]:
QSO_select=QSO[QSO['len_neo']>20]
#QSO_select=QSO_select[QSO_select['pearson_coeff_neo']>0.8]
QSO_select=drop_index(QSO_select)

In [34]:
QSO_select['log(L5100)']=(QSO_select['lg(LHalpha)']-np.log10(5.25)-42)/1.157+44
QSO_select['log(Lbol)']=np.log10(9)+QSO_select['log(L5100)']
QSO_select['log(Ledd)']=np.log10(1.263*10**38)+QSO_select['lg(L_Mbh)']
QSO_select['log(Lbol/Ledd)']=QSO_select['log(Lbol)']-QSO_select['log(Ledd)']
#print(QSO_select.columns)
#D_cm_QSO_select=cosmo.luminosity_distance(z=QSO_select['redshift']).to(u.cm)
QSO_select['D_cm']=cosmo.luminosity_distance(z=QSO_select['redshift']).to(u.cm)
QSO_select['logL_W1']= (-48.6-QSO_select['mean_m_w1_neo']-2.699)/2.5+np.log10(4*np.pi*QSO_select['D_cm']**2)+np.log10((constants.c/(3.4*u.um)).to(u.Hz).value)
QSO_select['logL_W2']= (-48.6-QSO_select['mean_m_w2_neo']-3.339)/2.5+np.log10(4*np.pi*QSO_select['D_cm']**2)+np.log10((constants.c/(4.6*u.um)).to(u.Hz).value)

In [ ]:
QSO_select.to_csv('%s/QSO_select_wise_var.csv'%CLAGN_statistic_plot_dir,index=False)

QSO.to_csv('%s/QSO_wise_var.csv'%CLAGN_statistic_plot_dir)

In [59]:
len(QSO[QSO['len_neo']>2]),len(QSO[QSO['len_neo']>10]),len(QSO[QSO['len_neo']>20])

(857, 762, 732)

In [ ]:
10**(np.mean(QSO_select['log(Lbol/Ledd)']))

In [ ]:
10**(np.mean(QSO_select['log(Lbol/Ledd)']))/10**(np.mean(QSO_select['logL_W1']-np.log10(1.263*10**38)-QSO_select['lg(L_Mbh)']))

## def get_minmax_mag_name_one

In [35]:
def get_minmax_mag_name_one(name_test,data_test_neo,data_test_mep):    
    maxmag=0
    minmag=15
    maxmag_var=0 
    
    if len(data_test_neo)>20:    
        mean_mjds_neo,mean_mags_neo,mean_mags_e_neo=median_lc(data_test_neo,'mjd','w1mpro','w1sigmpro',interval=180,)
        maxmag=max(max(mean_mags_neo),maxmag)  
        minmag=min(min(mean_mags_neo),minmag)  
    
    if len(data_test_mep)>5: 
        mean_mjds_mep,mean_mags_mep,mean_mags_e_mep=median_lc(data_test_mep,'mjd','w1mpro_ep','w1sigmpro_ep',interval=180,)
        maxmag=max(max(mean_mags_mep),maxmag)  
        minmag=min(min(mean_mags_mep),minmag) 
        
    maxmag_var=maxmag-minmag
    return maxmag_var           

In [ ]:
wisename_varlists5=[]
for index,name_test in enumerate(CLAGN_210919['Name']): 
    #print(name_test)
    search_name=name_test.replace(' ','') 
    save_name=search_name.strip()
    data_test_neo,data_test_mep= get_wise_data(name_test,datawise_dir='/Users/brettlv/blog/brettlv.github.io/pythoncode/changinglookAGN/CLAGN_wise_data/')
    maxmag_var=get_minmax_mag_name_one(name_test,data_test_neo,data_test_mep)
    
    if maxmag_var > 0.5:
        print(save_name,'%.2f'%maxmag_var,CLAGN_210919.loc[index,'OX'])
        wisename_varlists5.append(save_name)

In [ ]:
len(wisename_varlists5)

In [ ]:
wisename_varlists6=[]
for index,name_test in enumerate(CLAGN_210919['Name']): 
    #print(name_test)
    search_name=name_test.replace(' ','') 
    save_name=search_name.strip()
    data_test_neo,data_test_mep= get_wise_data(name_test,datawise_dir='/Users/brettlv/blog/brettlv.github.io/pythoncode/changinglookAGN/CLAGN_wise_data/')
    maxmag_var=get_minmax_mag_name_one(name_test,data_test_neo,data_test_mep)
    
    if maxmag_var > 0.6:
        print(save_name,'%.2f'%maxmag_var,CLAGN_210919.loc[index,'OX'])
        wisename_varlists6.append(save_name)

In [ ]:
len(wisename_varlists6)

In [ ]:
wisename_varlists10=[]
for index,name_test in enumerate(CLAGN_210919['Name']): 
    #print(name_test)
    search_name=name_test.replace(' ','') 
    save_name=search_name.strip()
    data_test_neo,data_test_mep= get_wise_data(name_test,datawise_dir='/Users/brettlv/blog/brettlv.github.io/pythoncode/changinglookAGN/CLAGN_wise_data/')
    maxmag_var=get_minmax_mag_name_one(name_test,data_test_neo,data_test_mep)
    
    if maxmag_var > 1.0:
        print(save_name,'%.2f'%maxmag_var,CLAGN_210919.loc[index,'OX'])
        wisename_varlists10.append(save_name)

## def get_rebinwise_data

In [36]:
def get_rebinwise_data(data_test_neo,data_test_mep,sigma_outlier=10):
    #print(name_test)
    #search_name=name_test.replace(' ','') 
    #save_name=search_name.strip()
    data_rebin_neo=pd.DataFrame([])
    data_rebin_mep=pd.DataFrame([])
    
    timelabel,maglabel,e_maglabel=('mjd','w1mpro','w1sigmpro')
    band='W1'
    mean_mjds,mean_mags,mean_mags_e=median_lc(data_test_neo,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=sigma_outlier)

    data_rebin_neo['t_W1']=mean_mjds
    data_rebin_neo['W1']=mean_mags
    data_rebin_neo['e_W1']=mean_mags_e
    
    timelabel,maglabel,e_maglabel=('mjd','w2mpro','w2sigmpro')
    band='W2' 
    mean_mjds,mean_mags,mean_mags_e=median_lc(data_test_neo,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=sigma_outlier)
    
    data_rebin_neo['t_W2']=mean_mjds
    data_rebin_neo['W2']=mean_mags
    data_rebin_neo['e_W2']=mean_mags_e
    
#############################################################
    timelabel,maglabel,e_maglabel=('mjd','w1mpro_ep','w1sigmpro_ep')
    band='W1'
    mean_mjds,mean_mags,mean_mags_e=median_lc(data_test_mep,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=sigma_outlier)

    data_rebin_mep['t_W1']=mean_mjds
    data_rebin_mep['W1']=mean_mags
    data_rebin_mep['e_W1']=mean_mags_e
    
    timelabel,maglabel,e_maglabel=('mjd','w2mpro_ep','w2sigmpro_ep')
    band='W2' 
    mean_mjds,mean_mags,mean_mags_e=median_lc(data_test_mep,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=sigma_outlier)
    
    data_rebin_mep['t_W2']=mean_mjds
    data_rebin_mep['W2']=mean_mags
    data_rebin_mep['e_W2']=mean_mags_e
        
        
    return data_rebin_neo,data_rebin_mep
    

## plot_mag_

In [37]:
def plot_mag_w1(ax,data_test_neo,data_test_mep):
    if len(data_test_neo)>0:
        ax.errorbar(data_test_neo['mjd'],data_test_neo['w1mpro'],data_test_neo['w1sigmpro'],ls='',marker='.',color='black',label='W1',zorder=0)
    if len(data_test_mep)>0:    
        ax.errorbar(data_test_mep['mjd'],data_test_mep['w1mpro_ep'],data_test_mep['w1sigmpro_ep'],ls='',marker='.',color='black',label='W1',zorder=0)
    #ax.scatter(data_test_neo_w1_outlier['mjd'],data_test_neo_w1_outlier['w1mpro'],marker='.',color='red',zorder=1)
    #ax.scatter(data_test_mep_w1_outlier['mjd'],data_test_mep_w1_outlier['w1mpro_ep'],marker='.',color='red',zorder=1)

def plot_mag_w2(ax,data_test_neo,data_test_mep):
    if len(data_test_neo)>0:
        ax.errorbar(data_test_neo['mjd'],data_test_neo['w2mpro'],data_test_neo['w2sigmpro'],ls='',marker='.',color='black',label='W2',zorder=0)
    if len(data_test_mep)>0:
        ax.errorbar(data_test_mep['mjd'],data_test_mep['w2mpro_ep'],data_test_mep['w2sigmpro_ep'],ls='',marker='.',color='black',label='W2',zorder=0)
    #ax.scatter(data_test_neo_w2_outlier['mjd'],data_test_neo_w2_outlier['w2mpro'],marker='.',color='red',zorder=1)
    #ax.scatter(data_test_mep_w2_outlier['mjd'],data_test_mep_w2_outlier['w2mpro_ep'],marker='.',color='red',zorder=1)
    
def plot_rebinmag_w1(ax,data_rebin_neo,data_rebin_mep):
    if len(data_rebin_neo)>0:
        ax.errorbar(data_rebin_neo['t_W1'],data_rebin_neo['W1'],data_rebin_neo['e_W1'],ls='',marker='^',color='b',label='W1',zorder=2)
    if len(data_rebin_mep)>0:
        ax.errorbar(data_rebin_mep['t_W1'],data_rebin_mep['W1'],data_rebin_mep['e_W1'],ls='',marker='^',color='b',label='W1',zorder=2)

def plot_rebinmag_w2(ax,data_rebin_neo,data_rebin_mep):  
    if len(data_rebin_neo)>0:
        ax.errorbar(data_rebin_neo['t_W2'],data_rebin_neo['W2'],data_rebin_neo['e_W2'],ls='',marker='^',color='b',label='W1',zorder=2)
    if len(data_rebin_mep)>0:
        ax.errorbar(data_rebin_mep['t_W2'],data_rebin_mep['W2'],data_rebin_mep['e_W2'],ls='',marker='^',color='b',label='W1',zorder=2)


## def wise_unbin_lc_outlier

### test

In [ ]:
name_test='2MASSJ14345529+5723449'
data_test_neo,data_test_mep=get_wise_data(name_test,datawise_dir='/Users/brettlv/blog/brettlv.github.io/pythoncode/changinglookAGN/CLAGN_wise_data/')
data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep)

In [ ]:
len(data_test_neo),len(data_test_mep)

In [ ]:
len(data_rebin_neo),len(data_rebin_mep)

In [38]:
def wise_unbin_lc_outlier(name_test,datawise_dir='/Users/brettlv/blog/brettlv.github.io/pythoncode/changinglookAGN/CLAGN_wise_data/'):
    data_test_neo,data_test_mep=get_wise_data(name_test,datawise_dir=datawise_dir)
    data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep)

    search_name=name_test.replace(' ','') 
    save_name=search_name.strip()  

    save_lc_img_path=os.path.join(save_lc_img_dir,'%s.png'%save_name)

    fig = plt.figure(figsize=(8,4))
    fig.subplots_adjust(hspace=0.0, wspace = 0.0)
    ax = fig.add_subplot(111)
    
    plot_mag_w1(ax,data_test_neo,data_test_mep)
    plot_rebinmag_w1(ax,data_rebin_neo,data_rebin_mep)
    plot_outliers_w1(ax,data_test_neo,data_test_mep)    
    
    
    #ax.scatter(data_test_neo['mjd'],data_test_neo['w2mpro'],color='blue',label='W2')
    #ax.scatter(data_test_mep['mjd'],data_test_mep['w2mpro_ep'],color='blue',label='W2')

    bottom, top = ax.set_ylim() 
    if bottom< top:
        ax.set_ylim(top,bottom)

    plt.xlabel('mjd')
    plt.ylabel(r'$mag$')

    ax.text(0.5, 0.9,name_test , horizontalalignment = 'center', verticalalignment = 'center',transform = ax.transAxes, fontsize = 10)
    plt.savefig(save_lc_img_path,dpi=400, transparent=False, bbox_inches='tight')
    plt.close()

In [ ]:
len(wisename_varlists5)

In [ ]:
#save_lc_img_dir=os.path.join(CLAGN_statistic_plot_dir,'WISE_lc','unbin_CLAGN','full_CLAGN_wise_lc')
save_lc_img_dir=os.path.join(CLAGN_statistic_plot_dir,'WISE_lc','unbin_CLAGN','CLAGN_wise_gt50_gt0.5')
for index,name_test in enumerate(wisename_varlists5) :
    print(name_test)
    wise_unbin_lc_outlier(name_test,datawise_dir=CLAGN_wisedata_dir)

## def wise_rebin_lc

In [39]:
def wise_rebin_lc(name_test,datawise_dir='/Users/brettlv/blog/brettlv.github.io/pythoncode/changinglookAGN/CLAGN_wise_data/'):
    data_test_neo,data_test_mep=get_wise_data(name_test,datawise_dir)
    data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep,sigma_outlier=10)
    
    search_name=name_test.replace(' ','') 
    save_name=search_name.strip()
    save_lc_img_path=os.path.join(save_lc_img_dir,'%s.png'%save_name)
    
    fig = plt.figure(figsize=(8,4))
    fig.subplots_adjust(hspace=0.0, wspace = 0.0)
    ax = fig.add_subplot(111)
    
    plot_rebinmag_w1(ax,data_rebin_neo,data_rebin_mep)
    #plot_rebinmag_w2(ax,data_rebin_neo,data_rebin_mep)
    #ax.scatter(data_test_neo['mjd'],data_test_neo['w1mpro'],color='red',label='W1')
    #ax.scatter(data_test_mep['mjd'],data_test_mep['w1mpro_ep'],color='red',label='W1')


    #ax.scatter(data_test_neo['mjd'],data_test_neo['w2mpro'],color='blue',label='W2')
    #ax.scatter(data_test_mep['mjd'],data_test_mep['w2mpro_ep'],color='blue',label='W2')


    bottom, top = ax.set_ylim() 
    if bottom< top:
        ax.set_ylim(top,bottom)

    plt.xlabel('mjd')
    plt.ylabel(r'$mag$')

    ax.text(0.5, 0.9,name_test , horizontalalignment = 'center', verticalalignment = 'center',transform = ax.transAxes, fontsize = 10)
    plt.savefig(save_lc_img_path,dpi=400, transparent=False, bbox_inches='tight')
    plt.close()

In [ ]:
save_lc_img_dir=os.path.join(CLAGN_statistic_plot_dir,'WISE_lc','rebin_CLAGN','rebin_var_gt0.2')
for index,name_test in enumerate(wisevar_p2) :
    #print(name_test)
    wise_rebin_lc(name_test,datawise_dir=CLAGN_wisedata_dir)

In [ ]:
save_lc_img_dir=os.path.join(CLAGN_statistic_plot_dir,'WISE_lc','rebin_CLAGN','rebin_gt50_gt0.5mag')
for index,name_test in enumerate(wisename_varlists5) :
    #print(name_test)
    wise_rebin_lc(name_test,datawise_dir=CLAGN_wisedata_dir)

## def_wise_mag_to_flux

In [40]:
def wise_mag_to_flux(data,maglabel_w1,maglabel_w2,e_maglabel_w1,e_maglabel_w2,D_cm):
    data['logL_W1']= (-48.6-data[maglabel_w1]-2.699)/2.5+np.log10(4*np.pi*D_cm**2)+np.log10((constants.c/(3.4*u.um)).to(u.Hz).value)
    data['logL_W2']= (-48.6-data[maglabel_w2]-3.339)/2.5+np.log10(4*np.pi*D_cm**2)+np.log10((constants.c/(4.6*u.um)).to(u.Hz).value)
    data['e_logL_W1'] =   data[e_maglabel_w1]/2.5
    data['e_logL_W2'] =   data[e_maglabel_w2]/2.5
    return data

In [41]:
def get_wiseflux_rebin(data_rebin_neo,data_rebin_mep,D_cm):
    maglabel_w1,maglabel_w2,e_maglabel_w1,e_maglabel_w2=('W1','W2','e_W1','e_W2')  
    data_rebinflux_neo=wise_mag_to_flux(data_rebin_neo,maglabel_w1,maglabel_w2,e_maglabel_w1,e_maglabel_w2,D_cm)
    data_rebinflux_mep=wise_mag_to_flux(data_rebin_mep,maglabel_w1,maglabel_w2,e_maglabel_w1,e_maglabel_w2,D_cm)
                                                                                                              
    return data_rebinflux_neo,data_rebinflux_mep

## def_get_lag_data

In [42]:
def get_lag_data(data,timelabel,fluxlabel_w1,e_fluxlabel_w1):
    
    datalag_w1=pd.DataFrame({'t':data[timelabel],'flux':data[fluxlabel_w1],\
                                  'err':data[e_fluxlabel_w1]},\
                                  index=None,columns=['t','flux','err'])
    return datalag_w1

In [43]:
def get_wise_lc(data_rebinflux_neo,data_rebinflux_mep):
    lc_w1= pd.concat([data_rebinflux_neo[['t_W1','logL_W1','e_logL_W1']],data_rebinflux_mep[['t_W1','logL_W1','e_logL_W1']]])
    lc_w2= pd.concat([data_rebinflux_neo[['t_W2','logL_W2','e_logL_W2']],data_rebinflux_mep[['t_W2','logL_W2','e_logL_W2']]])
    
    lc_w1=lc_w1.sort_values('t_W1')
    lc_w2=lc_w2.sort_values('t_W2')
    lc_w1=drop_index(lc_w1)
    lc_w2=drop_index(lc_w2)
    
    #return lc_w1,lc_w2
    datalag_w1=get_lag_data(lc_w1,'t_W1','logL_W1','e_logL_W1')
    datalag_w2=get_lag_data(lc_w2,'t_W2','logL_W2','e_logL_W2')
    return datalag_w1,datalag_w2

# workflow

In [ ]:
name_test='SDSS J153308.02+443208.4'
D_cm_J153308=CLAGN_210919_mass[CLAGN_210919_mass['Name']=='SDSS J153308.02+443208.4']['D_cm'].values[0]
print(name_test,D_cm_J153308)

data_test_neo,data_test_mep=get_wise_data(name_test,datawise_dir=CLAGN_wisedata_dir)
data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep)
print(data_rebin_neo.columns,data_rebin_mep.columns)
data_rebinflux_neo,data_rebinflux_mep=get_wiseflux_rebin(data_rebin_neo,data_rebin_mep,D_cm_J153308)
print(data_rebinflux_neo.columns,data_rebinflux_mep.columns)    

In [ ]:
datalag_w1,datalag_w2=get_wise_lc(data_rebinflux_neo,data_rebinflux_mep)

print(datalag_w1.columns,datalag_w2.columns)

In [ ]:
fig = plt.figure(figsize=(4,4))
fig.subplots_adjust(hspace=0.0, wspace = 0.0)
ax = fig.add_subplot(111)

plot_mag_w1(ax,data_test_neo,data_test_mep)
plot_rebinmag_w1(ax,data_rebin_neo,data_rebin_mep)
plot_outliers_w1(ax,data_test_neo,data_test_mep)
set_ax_tick(ax)

#set_ax_locator(ax,xma=2000,xmi=1000,yma=0.2,ymi=0.1)
ax.text(0.03, 0.85, name_test+' W1', fontsize = 15, transform = ax.transAxes)
bottom, top = ax.set_ylim() 
ax.set_ylim(top,bottom)
plt.show()
#fig.savefig('/Users/brettlv/Downloads/CLAGN_statistic_plot/WISE_lc/unbin_CLAGN/CLAGN_wise_gt50_gt0.5/W1_unbinlc_%s.png'%save_name,dpi=400,transparent=False,bbox_inches='tight')      
plt.close()

In [ ]:
fig = plt.figure(figsize=(4,4))
fig.subplots_adjust(hspace=0.0, wspace = 0.0)
ax = fig.add_subplot(111)

plot_mag_w2(ax,data_test_neo,data_test_mep)
plot_rebinmag_w2(ax,data_rebin_neo,data_rebin_mep)
plot_outliers_w2(ax,data_test_neo,data_test_mep)

set_ax_tick(ax)

#set_ax_locator(ax,xma=2000,xmi=1000,yma=0.2,ymi=0.1)
ax.text(0.03, 0.85, name_test+' W2', fontsize = 15, transform = ax.transAxes)
bottom, top = ax.set_ylim() 
ax.set_ylim(top,bottom)
plt.show()
#fig.savefig('/Users/brettlv/Downloads/CLAGN_statistic_plot/WISE_lc/unbin_CLAGN/CLAGN_wise_gt50_gt0.5/W1_unbinlc_%s.png'%save_name,dpi=400,transparent=False,bbox_inches='tight')      
plt.close()

In [ ]:
name_test='Mrk 1018'
data_test_neo,data_test_mep=get_wise_data(name_test,datawise_dir='/Users/brettlv/blog/brettlv.github.io/pythoncode/changinglookAGN/CLAGN_wise_data/')
data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep)

fig = plt.figure(figsize=(4,4))
fig.subplots_adjust(hspace=0.0, wspace = 0.0)
ax = fig.add_subplot(111)

plot_mag_w1(ax,data_test_neo,data_test_mep)
plot_rebinmag_w1(ax,data_rebin_neo,data_rebin_mep)
plot_outliers_w1(ax,data_test_neo,data_test_mep)
set_ax_tick(ax)

#set_ax_locator(ax,xma=2000,xmi=1000,yma=0.2,ymi=0.1)
ax.text(0.53, 0.85, name_test+' W1', fontsize = 15, transform = ax.transAxes)
bottom, top = ax.set_ylim() 
ax.set_ylim(top,bottom)
plt.show()
#fig.savefig('/Users/brettlv/Downloads/CLAGN_statistic_plot/WISE_lc/unbin_CLAGN/CLAGN_wise_gt50_gt0.5/W1_unbinlc_%s.png'%save_name,dpi=400,transparent=False,bbox_inches='tight')      
plt.close()

# get_rebin_alldata

In [ ]:
CLAGN_210919_mass['Name'][:5]

In [ ]:
rebin_alldata=pd.concat([],[])   

In [ ]:
CLAGN_210919_mass.columns

In [ ]:
Name t logL logLerr logM OX

In [ ]:
data = pd.DataFrame()
a = [[1,2,3],[4,5,6]]
data = data.append(a,ignore_index=True)
a = [[7,8,9],[10,11,12]]
data = data.append(a,ignore_index=True)
print(data)


In [ ]:
len(CLAGN_210919_mass)

## CLAGN

In [55]:
rebin_alldata_w1_only=pd.DataFrame([])
for index,name_test in enumerate(CLAGN_210919_mass['Name']):
    data_test_neo,data_test_mep=get_wise_data(name_test,datawise_dir=CLAGN_wisedata_dir)
    data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep)
    D_cm=CLAGN_210919_mass['D_cm'][index]
    logmass=CLAGN_210919_mass['lg(MBH/Msun)'][index]
    OX=CLAGN_210919_mass['OX'][index]
    
    data_rebinflux_neo,data_rebinflux_mep=get_wiseflux_rebin(data_rebin_neo,data_rebin_mep,D_cm)    
    datalag_w1,datalag_w2=get_wise_lc(data_rebinflux_neo,data_rebinflux_mep)
    #print(name_test,datalag_w1)
    rebin_alldata_w1=pd.DataFrame([])
    for j in range(len(datalag_w1)):
        rebin_alldata_w1.loc[j,'Name']=name_test
        rebin_alldata_w1.loc[j,'mjd']=datalag_w1['t'][j]
        rebin_alldata_w1.loc[j,'logL']=datalag_w1['flux'][j]
        rebin_alldata_w1.loc[j,'logLerr']=datalag_w1['err'][j]
        rebin_alldata_w1.loc[j,'D_cm']=D_cm
        rebin_alldata_w1.loc[j,'logM']=logmass
        rebin_alldata_w1.loc[j,'OX']=OX
        rebin_alldata_w1.loc[j,'band']='w1'
    rebin_alldata_w1_only= rebin_alldata_w1_only.append(rebin_alldata_w1,ignore_index=True)      
        
        

In [56]:
len(rebin_alldata_w1_only)

1165

In [44]:
rebin_alldata=pd.DataFrame([])

for index,name_test in enumerate(CLAGN_210919_mass['Name']):
    data_test_neo,data_test_mep=get_wise_data(name_test,datawise_dir=CLAGN_wisedata_dir)
    data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep)
    D_cm=CLAGN_210919_mass['D_cm'][index]
    logmass=CLAGN_210919_mass['lg(MBH/Msun)'][index]
    OX=CLAGN_210919_mass['OX'][index]
    
    data_rebinflux_neo,data_rebinflux_mep=get_wiseflux_rebin(data_rebin_neo,data_rebin_mep,D_cm)    
    datalag_w1,datalag_w2=get_wise_lc(data_rebinflux_neo,data_rebinflux_mep)
    
    rebin_alldata_w1=pd.DataFrame([])
    rebin_alldata_w2=pd.DataFrame([])

    for j in range(len(datalag_w1)):
        rebin_alldata_w1.loc[j,'Name']=name_test
        rebin_alldata_w1.loc[j,'mjd']=datalag_w1['t'][j]
        rebin_alldata_w1.loc[j,'logL']=datalag_w1['flux'][j]
        rebin_alldata_w1.loc[j,'logLerr']=datalag_w1['err'][j]
        rebin_alldata_w1.loc[j,'D_cm']=D_cm
        rebin_alldata_w1.loc[j,'logM']=logmass
        rebin_alldata_w1.loc[j,'OX']=OX
        rebin_alldata_w1.loc[j,'band']='w1'
     
        
    for k in range(len(datalag_w2)):
        rebin_alldata_w2.loc[k,'Name']=name_test
        rebin_alldata_w2.loc[k,'mjd']=datalag_w2['t'][k]
        rebin_alldata_w2.loc[k,'logL']=datalag_w2['flux'][k]
        rebin_alldata_w2.loc[k,'logLerr']=datalag_w2['err'][k]
        rebin_alldata_w2.loc[k,'D_cm']=D_cm
        rebin_alldata_w2.loc[k,'logM']=logmass
        rebin_alldata_w2.loc[k,'OX']=OX
        rebin_alldata_w2.loc[k,'band']='w2' 
        
       
    rebin_alldata= rebin_alldata.append(rebin_alldata_w1,ignore_index=True)  
    rebin_alldata= rebin_alldata.append(rebin_alldata_w2,ignore_index=True)  
         
 

In [45]:
rebin_alldata.to_csv('%s/CLAGN_210919_mass_rebin_w1_w2.csv'%CLAGN_statistic_plot_dir,index=False)

In [47]:
len(rebin_alldata)

2330

In [52]:
rebin_alldata_w1=rebin_alldata[rebin_alldata['band']=='w1']
rebin_alldata_w1=drop_index(rebin_alldata_w1)
rebin_alldata_w2=rebin_alldata[rebin_alldata['band']=='w2']
rebin_alldata_w2=drop_index(rebin_alldata_w2)

In [53]:
len(rebin_alldata_w1),len(rebin_alldata_w2)

(1165, 1165)

In [ ]:
rebin_alldata_A=rebin_alldata[rebin_alldata['OX']==' A ']
rebin_alldata_A=drop_index(rebin_alldata_A)
rebin_alldata_B=rebin_alldata[rebin_alldata['OX']==' B ']
rebin_alldata_B=drop_index(rebin_alldata_B)

for index,name_test in enumerate(rebin_alldata_A['Name']):
    rebin_alldata_A.loc[index,"Name_"]=rebin_alldata_A.loc[index,"Name"].replace(' ','').strip()

In [ ]:
rebin_alldata_A.columns

In [ ]:
rebin_alldata_A_w1=rebin_alldata_A[rebin_alldata_A['band']=='w1']
rebin_alldata_A_w1=drop_index(rebin_alldata_A_w1)
rebin_alldata_A_w2=rebin_alldata_A[rebin_alldata_A['band']=='w2']
rebin_alldata_A_w2=drop_index(rebin_alldata_A_w2)

In [ ]:
rebin_alldata_A_w1.columns

## CLAGN_210919_date

In [ ]:
CLAGN_210919_date=pd.read_excel('/Users/brettlv/blog/IHEP_sync/data/data_clagn/CLAGN_list210919.xlsx',sheet_name='Sheet2')
CLAGN_210919_date=CLAGN_210919_date[CLAGN_210919_date['select']==1]
CLAGN_210919_date=drop_index(CLAGN_210919_date)


In [ ]:
CLAGN_210919_date_type=pd.DataFrame([])

for index,name_test in enumerate(CLAGN_210919_date['Name']):
    print(name_test)
    data_I=rebin_alldata_A[rebin_alldata_A['Name_']==name_test.replace(' ','').strip()]
    data_I=drop_index(data_I)
    data_I['type']=np.nan
    
    if not np.isnan(CLAGN_210919_date.loc[index,'TypeduringWISE']):
        for index_,mjd_ in enumerate(data_I['mjd']):
            data_I.loc[index_,'type']=CLAGN_210919_date.loc[index,'TypeduringWISE']   
        #data_I['type']=CLAGN_210919_date.loc[index,'TypeduringWISE']
        print(name_test,CLAGN_210919_date.loc[index,'TypeduringWISE'])
        #CLAGN_210919_date_type=CLAGN_210919_date_type.append(data_I,ignore_index=True)
        #print(data_I)
    else:
        for index_,mjd_ in enumerate(data_I['mjd']):
            if not np.isnan(CLAGN_210919_date.loc[index,'Type1_mjd']):
                if abs(mjd_-CLAGN_210919_date.loc[index,'Type1_mjd'])<=90:
                    data_I.loc[index_,'type']=1
                                            
            if not np.isnan(CLAGN_210919_date.loc[index,'Type1_mjd_start']) :
                if not np.isnan(CLAGN_210919_date.loc[index,'Type1_mjd_end']):
                    if mjd_ >= CLAGN_210919_date.loc[index,'Type1_mjd_start'] and mjd_<= CLAGN_210919_date.loc[index,'Type1_mjd_end']:
                        data_I.loc[index_,'type']=1                                                                                       
                else:
                    if abs(mjd_-CLAGN_210919_date.loc[index,'Type1_mjd_start'])<=90:
                        data_I.loc[index_,'type']=1
                                                                                               
            if not np.isnan(CLAGN_210919_date.loc[index,'Type2_mjd']):
                if abs(mjd_-CLAGN_210919_date.loc[index,'Type2_mjd'])<=90:
                    data_I.loc[index_,'type']=2
                                            
            if not np.isnan(CLAGN_210919_date.loc[index,'Type2_mjd_start']) :
                if not np.isnan(CLAGN_210919_date.loc[index,'Type2_mjd_end']):
                    if mjd_ >= CLAGN_210919_date.loc[index,'Type2_mjd_start'] and mjd_<= CLAGN_210919_date.loc[index,'Type2_mjd_end']:
                        data_I.loc[index_,'type']=2                                                                                       
                else:
                    if abs(mjd_-CLAGN_210919_date.loc[index,'Type2_mjd_start'])<=90: 
                        data_I.loc[index_,'type']=2  
                        
    CLAGN_210919_date_type=CLAGN_210919_date_type.append(data_I,ignore_index=True)              

    


In [ ]:
CLAGN_210919_date_type=CLAGN_210919_date_type[CLAGN_210919_date_type['type'].notnull()]
CLAGN_210919_date_type['ledd']=CLAGN_210919_date_type['logL']-CLAGN_210919_date_type['logM']-np.log10(1.263e38)
CLAGN_210919_date_type.to_csv('%s/CLAGN_210919_date_type.csv'%CLAGN_statistic_plot_dir,index=False)

CLAGN_210919_date_type1=CLAGN_210919_date_type[CLAGN_210919_date_type['type']==1]
CLAGN_210919_date_type2=CLAGN_210919_date_type[CLAGN_210919_date_type['type']==2]

CLAGN_210919_date_type1=drop_index(CLAGN_210919_date_type1)
CLAGN_210919_date_type2=drop_index(CLAGN_210919_date_type2)

In [ ]:
len(CLAGN_210919_date_type),len(CLAGN_210919_date_type1),len(CLAGN_210919_date_type2)

In [ ]:
CLAGN_210919_date_type2

In [ ]:
CLAGN_210919_date_type1

## Gu09

In [ ]:
LLAGN_gu2009.columns

In [ ]:
#rebin_alldata_w2=pd.DataFrame([])
rebin_alldata_LLAGN_gu2009=pd.DataFrame([])

for index,name_test in enumerate(LLAGN_gu2009['Name']):
    data_test_neo,data_test_mep=get_LLAGNwise_data(name_test,datawise_dir=LLAGN_wisedata_dir)
    data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep)
    D_cm=LLAGN_gu2009['D_cm'][index]
    logmass=LLAGN_gu2009['∗ log MBH'][index]
    #OX=CLAGN_210919_mass['OX'][index]
    data_rebinflux_neo,data_rebinflux_mep=get_wiseflux_rebin(data_rebin_neo,data_rebin_mep,D_cm)    
    datalag_w1,datalag_w2=get_wise_lc(data_rebinflux_neo,data_rebinflux_mep)
    rebin_alldata_w1_LLAGN_gu2009=pd.DataFrame([])
    
    for j in range(len(datalag_w1)):
        rebin_alldata_w1_LLAGN_gu2009.loc[j,'Name']=name_test
        rebin_alldata_w1_LLAGN_gu2009.loc[j,'mjd']=datalag_w1['t'][j]
        rebin_alldata_w1_LLAGN_gu2009.loc[j,'logL']=datalag_w1['flux'][j]
        rebin_alldata_w1_LLAGN_gu2009.loc[j,'logLerr']=datalag_w1['err'][j]
        rebin_alldata_w1_LLAGN_gu2009.loc[j,'D_cm']=D_cm
        rebin_alldata_w1_LLAGN_gu2009.loc[j,'logM']=logmass
           
    rebin_alldata_LLAGN_gu2009= rebin_alldata_LLAGN_gu2009.append(rebin_alldata_w1_LLAGN_gu2009,ignore_index=True)   
         
 

In [ ]:
rebin_alldata_LLAGN_gu2009.to_csv('%s/LLAGN_rebin_w1.csv'%CLAGN_statistic_plot_dir,index=False)

In [ ]:
len(rebin_alldata_LLAGN_gu2009)

## QSO_select

In [ ]:
QSO_select.columns

In [57]:
len(QSO_select)

732

In [ ]:
name_test=QSO_select['Name'][21]
data_test_neo,data_test_mep=get_QSOwise_data(name_test,datawise_dir=QSO_wisedata_dir)

In [ ]:
data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep,sigma_outlier=10)

In [ ]:
#rebin_alldata_w2=pd.DataFrame([])
rebin_alldata_QSO_select=pd.DataFrame([])

for index,name_test in enumerate(QSO_select['Name']):
    data_test_neo,data_test_mep=get_QSOwise_data(name_test,datawise_dir=QSO_wisedata_dir)

    #print(index)
    data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep,sigma_outlier=10)

    D_cm=QSO_select['D_cm'][index]
    logmass=QSO_select['lg(L_Mbh)'][index]
    #OX=CLAGN_210919_mass['OX'][index]
    data_rebinflux_neo,data_rebinflux_mep=get_wiseflux_rebin(data_rebin_neo,data_rebin_mep,D_cm)    
    datalag_w1,datalag_w2=get_wise_lc(data_rebinflux_neo,data_rebinflux_mep)
    
    rebin_alldata_w1_QSO_select=pd.DataFrame([])

    for j in range(len(datalag_w1)):
        rebin_alldata_w1_QSO_select.loc[j,'Name']=name_test
        rebin_alldata_w1_QSO_select.loc[j,'mjd']=datalag_w1['t'][j]
        rebin_alldata_w1_QSO_select.loc[j,'logL']=datalag_w1['flux'][j]
        rebin_alldata_w1_QSO_select.loc[j,'logLerr']=datalag_w1['err'][j]
        rebin_alldata_w1_QSO_select.loc[j,'D_cm']=D_cm
        rebin_alldata_w1_QSO_select.loc[j,'logM']=logmass

    rebin_alldata_QSO_select= rebin_alldata_QSO_select.append(rebin_alldata_w1_QSO_select,ignore_index=True)   

 

In [ ]:
rebin_alldata_QSO_select.to_csv('%s/QSO_select_rebin_w1.csv'%CLAGN_statistic_plot_dir,index=False)

In [ ]:
len(rebin_alldata_QSO_select)

# writedata

## write_lagfile

In [ ]:
def write_lagfile(rootdir,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax):
    if not os.path.exists(os.path.join(rootdir,sourcename)):
        os.mkdir(os.path.join(rootdir,sourcename))
        
    data1= dataname1[dataname1['t']>rangemin]
    data1= data1[data1['t']<rangemax]   
    path_data1=os.path.join(rootdir,sourcename,labelname1+'.csv')
    data1.to_csv(path_data1,index=False,header=False,sep=',')
    
    data2= dataname2[dataname2['t']>rangemin]
    data2= data2[data2['t']<rangemax]    
    path_data2=os.path.join(rootdir,sourcename,labelname2+'.csv')
    data2.to_csv(path_data2,index=False,header=False,sep=',')  
    return path_data1,path_data2    

## get_lagfile_path

In [ ]:
def get_lagfile(rootdir,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax):
    
    #data1= dataname1[dataname1['t']>rangemin]
    #data1= data1[data1['t']<rangemax]   
    path_data1=os.path.join(rootdir,sourcename,labelname1+'.csv')
    #data1.to_csv(path_data1,index=False,header=False,sep=',')
    
    #data2= dataname2[dataname2['t']>rangemin]
    #data2= data2[data2['t']<rangemax]    
    path_data2=os.path.join(rootdir,sourcename,labelname2+'.csv')
    #data2.to_csv(path_data2,index=False,header=False,sep=',')  
    return path_data1,path_data2 

# def_run ccf

In [ ]:
def run_ccf(rootdir,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel=-10,lagrangeh=30):
    #command='python ccf_ngc1566_cpython.py %s %s %s %s %d %d %d %d'%(path_data1,path_data2,labelname1,labelname2,rangemin,rangemax,lagrangel,lagrangeh)
    os.chdir(rootdir)
    source_path=os.path.join(rootdir,sourcename)
    path_data1,path_data2=write_lagfile(rootdir,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax)
    command='python2 CCF_cpython.py %s %s %d %d %d %d %s'%(path_data1,path_data2,rangemin,rangemax,lagrangel,lagrangeh,source_path)
    print(command)
    os.system(command)  

In [ ]:
rootdir_ccf='/Users/brettlv/Downloads/CCF/'
if not os.path.exists(rootdir_ccf):
    os.mkdir(rootdir_ccf)
    print(rootdir_ccf)

## def_plot_ccf

In [ ]:
def plot_ccf(rootdir,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel=-10,lagrangeh=30):
    #command='python ccf_ngc1566_cpython.py %s %s %s %s %d %d %d %d'%(path_data1,path_data2,labelname1,labelname2,rangemin,rangemax,lagrangel,lagrangeh)
    os.chdir(rootdir)
    source_path=os.path.join(rootdir,sourcename)
    path_data1,path_data2=write_lagfile(rootdir,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax)
    command='python2 CCF_plot.py %s %s %d %d %d %d %s'%(path_data1,path_data2,rangemin,rangemax,lagrangel,lagrangeh,source_path)
    print(command)
    os.system(command)  

## def_plot_ccf_injupyter

In [ ]:
import matplotlib.pyplot as plt
def plot_ccf_injupyter(rootdir,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel, lagrangeh):
    lc1,lc2=get_lagfile(rootdir,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax)

    time_min,time_max=rangemin,rangemax
    lag_range = [lagrangel, lagrangeh]
    lc1label=lc1.split('/')[-1][:-4]
    lc2label=lc2.split('/')[-1][:-4]

    data1=np.loadtxt(lc1,  comments='#', delimiter=',')
    data2=np.loadtxt(lc2,  comments='#', delimiter=',')
    mjd1, flux1, err1 =data1[:,0],data1[:,1],data1[:,2]
    mjd2, flux2, err2 =data2[:,0],data2[:,1],data2[:,2]



    mjd1, flux1, err1=get_new_range_datas(mjd1, flux1, err1,time_min,time_max)
    mjd2, flux2, err2=get_new_range_datas(mjd2, flux2, err2,time_min,time_max)

    df1=pd.read_csv('%s/%s_%s_lag_r.csv'%(os.path.join(rootdir,sourcename),lc1label,lc2label))
    lag,r=df1['lag'],df1['r']

    df2=pd.read_csv('%s/%s_%s_lag_cent_peak.csv'%(os.path.join(rootdir,sourcename),lc1label,lc2label))

    tlags_centroid,tlags_peak=df2['tlags_centroid'],df2['tlags_peak']


    maxindex=np.argmax(r)
    Tmax=lag[maxindex]
    minindex=np.argmin(r)
    Tmin=lag[minindex]

    ###Calculate the best peak and centroid and their uncertainties using the median of the
    ##distributions.
    perclim = 84.1344746
    centau = stats.scoreatpercentile(tlags_centroid, 50)
    centau_uperr = (stats.scoreatpercentile(tlags_centroid, perclim))-centau
    centau_loerr = centau-(stats.scoreatpercentile(tlags_centroid, (100.-perclim)))

    peaktau = stats.scoreatpercentile(tlags_peak, 50)
    peaktau_uperr = (stats.scoreatpercentile(tlags_peak, perclim))-centau
    peaktau_loerr = centau-(stats.scoreatpercentile(tlags_peak, (100.-perclim)))

    ##########################################
    #Write results out to a file in case we want them later.
    ##########################################


    fig = plt.figure(figsize=(9,9))
    fig.subplots_adjust(hspace=0.4, wspace = 0.15)

    #Plot lightcurves
    ax1 = fig.add_subplot(3, 1, 1)
    ax1.errorbar(mjd1, flux1, yerr = err1, marker = 'o', linestyle = ':', color = 'r', label = 'LC 1 (%s)'%lc1label)
    
    ax1_2 = fig.add_subplot(3, 1, 2, sharex = ax1)
    ax1_2.errorbar(mjd2, flux2, yerr = err2, marker = '*', linestyle = ':', color = 'b', label = 'LC 2 (%s)'%lc2label)

    ax1.text(0.025, 0.825, lc1label, fontsize = 15, transform = ax1.transAxes)
    ax1_2.text(0.025, 0.825, lc2label, fontsize = 15, transform = ax1_2.transAxes)
    ax1.set_ylabel('LC 1 Flux')
    ax1_2.set_ylabel('LC 2 Flux')
    ax1_2.set_xlabel('MJD')
    #ax1.semilogy()
    #ax1_2.semilogy()

    #Plot CCF Information
    #xmin, xmax = -99, 99
    xmin, xmax = lag_range[0],lag_range[1]

    ax2 = fig.add_subplot(3, 3, 7)
    ax2.set_ylabel('CCF r')

    #ax2.axhline(sigmode,color='black',ls='--',lw=1,)
    ax2.axvline(Tmin,color='black',ls='--',lw=1,)
    ax2.axvline(Tmax,color='black',ls='--',lw=1,)
    ax2.text(0.2, 0.85, 'CCF ', horizontalalignment = 'center', verticalalignment = 'center', transform = ax2.transAxes, fontsize = 16)
    ax2.text(0.3, 0.25, 'max r %.2f '%max(r), horizontalalignment = 'center', verticalalignment = 'center', transform = ax2.transAxes, fontsize = 6)
    ax2.text(0.3, 0.05, 'min r %.2f '%min(r), horizontalalignment = 'center', verticalalignment = 'center', transform = ax2.transAxes, fontsize = 6)
    #ax2.text(0.3, 0.45, r'$p$-value %.1e '%pval, horizontalalignment = 'center', verticalalignment = 'center', transform = ax2.transAxes, fontsize = 6)


    ax2.set_xlim(xmin, xmax)
    ax2.set_ylim(-1.0, 1.0)
    ax2.plot(lag, r, color = 'k')
    set_ax_locator(ax2,xma=50,xmi=25,yma=0.4,ymi=0.2)
    
    '''ax2_2 = fig.add_subplot(4, 2, 7)
        ax2_2.set_ylabel('p_value')
        ax2_2.axvline(Tmin,color='black',ls='--',lw=1,)
        ax2_2.axvline(Tmax,color='black',ls='--',lw=1,)
        #ax2.text(0.2, 0.85, 'CCF ', horizontalalignment = 'center', verticalalignment = 'center', transform = ax2.transAxes, fontsize = 16)
        #ax2.text(0.2, 0.25, 'max r %.2f '%max(r), horizontalalignment = 'center', verticalalignment = 'center', transform = ax2.transAxes, fontsize = 6)
        #ax2.text(0.2, 0.05, 'min r %.2f '%min(r), horizontalalignment = 'center', verticalalignment = 'center', transform = ax2.transAxes, fontsize = 6)
        ax2_2.set_xlim(xmin, xmax)
        ax2.set_ylim(-1.0, 1.0)
        ax2_2.plot(tlags_peak, pvals, color = 'k',ls='',marker='.')'''


    ax3 = fig.add_subplot(3, 3, 8, sharex = ax2)
    ax3.set_xlim(xmin, xmax)
    ax3.axes.get_yaxis().set_ticks([])
    ax3.set_xlabel('Centroid Lag: %5.1f (+%5.1f -%5.1f)\n'%(centau, centau_uperr, centau_loerr)+'Peak, errors:%5.1f(+%5.1f-%5.1f) days'%(peaktau, peaktau_uperr, peaktau_loerr), fontsize = 15)

    ax3.text(0.2, 0.85, 'CCCD ', horizontalalignment = 'center', verticalalignment = 'center', transform = ax3.transAxes, fontsize = 16)
    n, bins, etc = ax3.hist(tlags_centroid, bins = 100, color = 'grey')
    
    ax3.axvline(centau,ls='-',lw=1,)
    ax3.axvline(centau+centau_uperr,ls='--',lw=1,)
    ax3.axvline(centau-centau_loerr,ls='--',lw=1,)
    
    ax4 = fig.add_subplot(3, 3, 9, sharex = ax2)
    ax4.set_ylabel('N')
    ax4.yaxis.tick_right()
    ax4.yaxis.set_label_position('right')
    #ax4.set_xlabel('Lag (days)')
    ax4.set_xlim(xmin, xmax)
    ax4.text(0.2, 0.85, 'CCPD ', horizontalalignment = 'center', verticalalignment = 'center', transform = ax4.transAxes, fontsize = 16)
    ax4.hist(tlags_peak, bins = bins, color = 'grey')
    ax4.axvline(peaktau,ls='-',lw=1,)
    ax4.axvline(peaktau+peaktau_uperr,ls='--',lw=1,)
    ax4.axvline(peaktau-peaktau_loerr,ls='--',lw=1,)
    
    set_ax_tick(ax1)
    set_ax_tick(ax1_2)
    set_ax_tick(ax2) 
    
    set_ax_tick(ax3) 
    set_ax_tick(ax4) 
    
    plt.show()
    #plt.savefig('%s/%s_%s_%s_ccf_%s_VS_%s_Centroid,error%10.3f(+%10.3f-%10.3f)_lagrange_%d_%d_d.png'%(rootdir,sourcename,int(time_min),int(time_max),lc1label,lc2label,centau, centau_uperr,centau_loerr,int(lag_range[0]),int(lag_range[1])), format = 'png', orientation = 'landscape', bbox_inches = 'tight')
    plt.close(fig)

# def run_javelin

log$\sigma$  DRW model amplitude (sigma)   
log$\tau$  timescale (tau)     
$\alpha$  
t   timelag  
w   width  
s   A scale factor  


These will correspond to the DRW model amplitude (sigma) and timescale (tau), same as for all of the previous examples on this page

Thus, for spectroscopic RM we have two parameters for the continuum DRW model, sigma and tau, and three parameters for each emission line model --- the lag t, the width of the tophat smoothing function w, and the flux scaling factor s; for photometric RM we have an additional parameter alpha describing the ratio between the two continua, one off and one on the line flux.

In [ ]:
def write_javelin_lagfile(rootdir,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax):
    if not os.path.exists(os.path.join(rootdir,sourcename)):
        os.mkdir(os.path.join(rootdir,sourcename))
        
    data1= dataname1[dataname1['t']>rangemin]
    data1= data1[data1['t']<rangemax]   
    path_data1=os.path.join(rootdir,sourcename,labelname1+'.csv')
    data1.to_csv(path_data1,index=False,header=False,sep=' ')
    
    data2= dataname2[dataname2['t']>rangemin]
    data2= data2[data2['t']<rangemax]    
    path_data2=os.path.join(rootdir,sourcename,labelname2+'.csv')
    data2.to_csv(path_data2,index=False,header=False,sep=' ')  
    return path_data1,path_data2    

In [ ]:
def run_javelin(rootdir_javelin,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel=-10,lagrangeh=30,widthl=0,widthh=200):
    #command='python ccf_ngc1566_cpython.py %s %s %s %s %d %d %d %d'%(path_data1,path_data2,labelname1,labelname2,rangemin,rangemax,lagrangel,lagrangeh)
    os.chdir(rootdir_javelin)
    source_path=os.path.join(rootdir_javelin,sourcename)
    path_data1,path_data2=write_javelin_lagfile(rootdir_javelin,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax)
    command='python2 javelin_spec.py %s %s %d %d %d %d %d %d %s'%(path_data1,path_data2,rangemin,rangemax,lagrangel,lagrangeh,widthl,widthh,source_path)
    print(command)
    os.system(command)  

In [ ]:
rootdir_javelin='/Users/brettlv/Downloads/Javelin/'
if not os.path.exists(rootdir_javelin):
    os.mkdir(rootdir_javelin)
    print(rootdir_javelin)

## def plot_javelin

In [ ]:
import matplotlib.pyplot as plt
from astropy.visualization import hist

def plot_javelin(rootdir,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel,lagrangeh):
    lc1,lc2=get_lagfile(rootdir,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax)

    time_min,time_max=rangemin,rangemax
    lag_range = [lagrangel, lagrangeh]
    lc1label=lc1.split('/')[-1][:-4]
    lc2label=lc2.split('/')[-1][:-4]

    data1=np.loadtxt(lc1,  comments='#', delimiter=' ')
    data2=np.loadtxt(lc2,  comments='#', delimiter=' ')
    mjd1, flux1, err1 =data1[:,0],data1[:,1],data1[:,2]
    mjd2, flux2, err2 =data2[:,0],data2[:,1],data2[:,2]


    mjd1, flux1, err1=get_new_range_datas(mjd1, flux1, err1,time_min,time_max)
    mjd2, flux2, err2=get_new_range_datas(mjd2, flux2, err2,time_min,time_max)



    tag = ""   
    #confchain = "dat/chain0.dat" + tag
    #topfchain = "dat/chain1.dat" + tag
    #doufchain = "dat/chain2.dat" + tag
    phofchain = "%s/%s/%s_mcmc_%s.dat"%(rootdir,sourcename,lc1label,lc2label) + tag
    javelin_lag_data=pd.read_csv(phofchain,header=None,sep=' ')
    lag_javelin=javelin_lag_data[2]
    width_javelin_data=javelin_lag_data[3]
    

    ###Calculate the best peak and centroid and their uncertainties using the median of the
    ##distributions.
    perclim = 84.1344746
    centau = stats.scoreatpercentile(lag_javelin, 50)
    centau_uperr = (stats.scoreatpercentile(lag_javelin, perclim))-centau
    centau_loerr = centau-(stats.scoreatpercentile(lag_javelin, (100.-perclim)))

    width_javelin = stats.scoreatpercentile(width_javelin_data, 50)
    width_javelin_uperr = (stats.scoreatpercentile(width_javelin_data, perclim))-width_javelin
    width_javelin_loerr = width_javelin-(stats.scoreatpercentile(width_javelin_data, (100.-perclim)))

    ##########################################
    #Write results out to a file in case we want them later.
    ##########################################


    fig = plt.figure(figsize=(8,6))
    fig.subplots_adjust(hspace=0.4, wspace = 0.15)

    #Plot lightcurves
    ax1 = fig.add_subplot(2, 1, 1)
    ax1.errorbar(mjd1, flux1, yerr = err1, marker = 'o', linestyle = ':', color = 'k', label = 'LC 1 (%s)'%lc1label)
    
    ax1_2 = ax1.twinx()
    ax1_2.errorbar(mjd2, flux2, yerr = err2, marker = '*', linestyle = ':', color = 'red', label = 'LC 2 (%s)'%lc2label)

    #ax1.text(0.025, 0.825, lc1label, fontsize = 15, transform = ax1.transAxes)
    #ax1_2.text(0.025, 0.825, lc2label, fontsize = 15, transform = ax1_2.transAxes)
    ax1.set_ylabel('LC 1 Flux')
    ax1_2.set_ylabel('LC 2 Flux')
    ax1.set_xlabel('MJD')
    #ax1.semilogy()
    #ax1_2.semilogy()

    #Plot CCF Information
    #xmin, xmax = -99, 99
    xmin, xmax = lag_range[0],lag_range[1]

    ax2 = fig.add_subplot(2,2,3)
    
    ax2.axvline(centau,color='black',ls='-',lw=1,)
    ax2.axvline(centau+centau_uperr,color='black',ls='--',lw=1,)
    ax2.axvline(centau-centau_loerr,color='black',ls='--',lw=1,)
        
   
     #ax2.set_ylim(0.0, 1.0)
    
    hist(lag_javelin,ax=ax2,bins=100,color='grey',density=True,zorder=0)
    
    ax2_1= fig.add_subplot(2, 2, 4)
    ax2_2=ax2_1.twinx()
    
    ax2_2.axvline(width_javelin,color='black',ls='-',lw=1,)
    ax2_2.axvline(width_javelin+width_javelin_uperr,color='black',ls='--',lw=1,)
    ax2_2.axvline(width_javelin-width_javelin_loerr,color='black',ls='--',lw=1,)
    hist(width_javelin_data,ax=ax2_2,bins=100,color='grey',density=True,zorder=0)

    
    ax1.legend(loc='upper left')
    ax1_2.legend(loc='upper right')
    
    ax2.set_xlim(xmin, xmax)
    ax2_2.set_xlim(min(width_javelin_data),max(width_javelin_data))
    
    ax2_1.yaxis.set_tick_params(which='major', size=10, width=2, direction='in',labelsize=0)
    ax2_1.yaxis.set_tick_params(which='minor', size=10, width=2, direction='in',labelsize=0)
    
    ax2.set_xlabel('Centroid Lag: %5.1f (+%5.1f -%5.1f)\n'%(centau, centau_uperr, centau_loerr), fontsize = 15)
    ax2_1.set_xlabel('Centroid width: %5.1f (+%5.1f -%5.1f)\n'%(width_javelin, width_javelin_uperr, width_javelin_loerr),fontsize = 15)

    #set_ax_tick(ax1)
    #set_ax_tick(ax1_2)
    set_ax_tick(ax2) 
    set_ax_tick(ax2_2) 
    ax1.xaxis.set_tick_params(which='major', size=10, width=2, direction='in')
    ax1.xaxis.set_tick_params(which='minor', size=5, width=2, direction='in',)
    ax1.yaxis.set_tick_params(which='major', size=10, width=2, direction='in')
    ax1.yaxis.set_tick_params(which='minor', size=5, width=2, direction='in',)
    #set_ax_locator(ax2,xma=50,xmi=25,yma=0.4,ymi=0.2)
    #set_ax_locator(ax2_2,xma=50,xmi=25,yma=0.4,ymi=0.2)
   
    #plt.show()
    plt.savefig('%s/%s/hist_%s_%s_javelin_%s_VS_%s_Centroid,error%10.3f(+%10.3f-%10.3f)_lagrange_%d_%d_d.png'%(rootdir,sourcename,int(time_min),int(time_max),lc1label,lc2label,centau, centau_uperr,centau_loerr,int(lag_range[0]),int(lag_range[1])), format = 'png', orientation = 'landscape', bbox_inches = 'tight')
    plt.close(fig)

## def plot_javelin_injupyter

In [ ]:
import matplotlib.pyplot as plt
from astropy.visualization import hist

def plot_javelin_injupyter(rootdir,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel, lagrangeh):
    lc1,lc2=get_lagfile(rootdir,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax)

    time_min,time_max=rangemin,rangemax
    lag_range = [lagrangel, lagrangeh]
    lc1label=lc1.split('/')[-1][:-4]
    lc2label=lc2.split('/')[-1][:-4]

    data1=np.loadtxt(lc1,  comments='#', delimiter=' ')
    data2=np.loadtxt(lc2,  comments='#', delimiter=' ')
    mjd1, flux1, err1 =data1[:,0],data1[:,1],data1[:,2]
    mjd2, flux2, err2 =data2[:,0],data2[:,1],data2[:,2]


    mjd1, flux1, err1=get_new_range_datas(mjd1, flux1, err1,time_min,time_max)
    mjd2, flux2, err2=get_new_range_datas(mjd2, flux2, err2,time_min,time_max)



    tag = ""   
    #confchain = "dat/chain0.dat" + tag
    #topfchain = "dat/chain1.dat" + tag
    #doufchain = "dat/chain2.dat" + tag
    phofchain = "%s/%s/%s_mcmc_%s.dat"%(rootdir,sourcename,lc1label,lc2label) + tag
    javelin_lag_data=pd.read_csv(phofchain,header=None,sep=' ')
    lag_javelin=javelin_lag_data[2]
    width_javelin_data=javelin_lag_data[3]
    

    ###Calculate the best peak and centroid and their uncertainties using the median of the
    ##distributions.
    perclim = 84.1344746
    centau = stats.scoreatpercentile(lag_javelin, 50)
    centau_uperr = (stats.scoreatpercentile(lag_javelin, perclim))-centau
    centau_loerr = centau-(stats.scoreatpercentile(lag_javelin, (100.-perclim)))

    width_javelin = stats.scoreatpercentile(width_javelin_data, 50)
    width_javelin_uperr = (stats.scoreatpercentile(width_javelin_data, perclim))-width_javelin
    width_javelin_loerr = width_javelin-(stats.scoreatpercentile(width_javelin_data, (100.-perclim)))

    ##########################################
    #Write results out to a file in case we want them later.
    ##########################################


    fig = plt.figure(figsize=(8,6))
    fig.subplots_adjust(hspace=0.4, wspace = 0.15)

    #Plot lightcurves
    ax1 = fig.add_subplot(2, 1, 1)
    ax1.errorbar(mjd1, flux1, yerr = err1, marker = 'o', linestyle = ':', color = 'k', label = 'LC 1 (%s)'%lc1label)
    
    ax1_2 = ax1.twinx()
    ax1_2.errorbar(mjd2, flux2, yerr = err2, marker = '*', linestyle = ':', color = 'red', label = 'LC 2 (%s)'%lc2label)

    #ax1.text(0.025, 0.825, lc1label, fontsize = 15, transform = ax1.transAxes)
    #ax1_2.text(0.025, 0.825, lc2label, fontsize = 15, transform = ax1_2.transAxes)
    ax1.set_ylabel('LC 1 Flux')
    ax1_2.set_ylabel('LC 2 Flux')
    ax1.set_xlabel('MJD')
    #ax1.semilogy()
    #ax1_2.semilogy()

    #Plot CCF Information
    #xmin, xmax = -99, 99
    xmin, xmax = lag_range[0],lag_range[1]

    ax2 = fig.add_subplot(2,2,3)
    
    ax2.axvline(centau,color='black',ls='-',lw=1,)
    ax2.axvline(centau+centau_uperr,color='black',ls='--',lw=1,)
    ax2.axvline(centau-centau_loerr,color='black',ls='--',lw=1,)
        
   
     #ax2.set_ylim(0.0, 1.0)
    
    hist(lag_javelin,ax=ax2,bins=100,color='grey',density=True,zorder=0)
    
    ax2_1= fig.add_subplot(2, 2, 4)
    ax2_2=ax2_1.twinx()
    
    ax2_2.axvline(width_javelin,color='black',ls='-',lw=1,)
    ax2_2.axvline(width_javelin+width_javelin_uperr,color='black',ls='--',lw=1,)
    ax2_2.axvline(width_javelin-width_javelin_loerr,color='black',ls='--',lw=1,)
    hist(width_javelin_data,ax=ax2_2,bins=100,color='grey',density=True,zorder=0)

    
    ax1.legend(loc='upper left')
    ax1_2.legend(loc='upper right')
    
    ax2.set_xlim(xmin, xmax)
    ax2_2.set_xlim(min(width_javelin_data),max(width_javelin_data))
    
    ax2_1.yaxis.set_tick_params(which='major', size=10, width=2, direction='in',labelsize=0)
    ax2_1.yaxis.set_tick_params(which='minor', size=10, width=2, direction='in',labelsize=0)
    
    ax2.set_xlabel('Centroid Lag: %5.1f (+%5.1f -%5.1f)\n'%(centau, centau_uperr, centau_loerr), fontsize = 15)
    ax2_1.set_xlabel('Centroid width: %5.1f (+%5.1f -%5.1f)\n'%(width_javelin, width_javelin_uperr, width_javelin_loerr),fontsize = 15)

    #set_ax_tick(ax1)
    #set_ax_tick(ax1_2)
    set_ax_tick(ax2) 
    set_ax_tick(ax2_2) 
    ax1.xaxis.set_tick_params(which='major', size=10, width=2, direction='in')
    ax1.xaxis.set_tick_params(which='minor', size=5, width=2, direction='in',)
    ax1.yaxis.set_tick_params(which='major', size=10, width=2, direction='in')
    ax1.yaxis.set_tick_params(which='minor', size=5, width=2, direction='in',)
    #set_ax_locator(ax2,xma=50,xmi=25,yma=0.4,ymi=0.2)
    #set_ax_locator(ax2_2,xma=50,xmi=25,yma=0.4,ymi=0.2)
   
    plt.show()
    #plt.savefig('%s/%s_%s_%s_ccf_%s_VS_%s_Centroid,error%10.3f(+%10.3f-%10.3f)_lagrange_%d_%d_d.png'%(rootdir,sourcename,int(time_min),int(time_max),lc1label,lc2label,centau, centau_uperr,centau_loerr,int(lag_range[0]),int(lag_range[1])), format = 'png', orientation = 'landscape', bbox_inches = 'tight')
    plt.close(fig)

# Mrk6

In [ ]:
Monitoring_AGNs_Du=pd.read_csv('/Users/brettlv/blog/IHEP_sync/data/Mrk6_lc_data/Mrk6.txt',delimiter='\s+',header=None)
Monitoring_AGNs_Du.columns=['Na','me','jd','F5100','e_F5100','FHb','e_FHb']

Monitoring_AGNs_Du['Name']=Monitoring_AGNs_Du['Na']+Monitoring_AGNs_Du['me']

Mrk6=Monitoring_AGNs_Du[Monitoring_AGNs_Du['Name']=='Mrk6']

Mrk6['mjd']=Mrk6['jd']+2457700-2400000.5

Mrk6=drop_index(Mrk6)

In [ ]:
Mrk6.columns

## plot_5100A_Hbeta

In [ ]:
fig = plt.figure(figsize=(6,4))
fig.subplots_adjust(hspace=0.0, wspace = 0.0)
ax = fig.add_subplot(111)

plotdata=Mrk6
ax.scatter(plotdata['mjd'],plotdata['F5100'],color='red',label='5100')
ax.scatter(plotdata['mjd'],plotdata['FHb'],color='black',label='Hbeta')

#ax.scatter(data_test_neo['mjd'],data_test_neo['w2mpro'],color='blue',label='W2')
#ax.scatter(data_test_mep['mjd'],data_test_mep['w2mpro_ep'],color='blue',label='W2')
handles, labels = ax.get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys(),bbox_to_anchor=(0.8, 0.9),ncol=1,
      loc=2,fontsize=10)

#bottom, top = plt.ylim() 
#plt.ylim(top,bottom)
set_ax_tick(ax)
plt.xlabel('mjd')
plt.ylabel(r'$flux$')
#ax.text(0.05, 0.9,name_test , horizontalalignment = 'center', verticalalignment = 'center',transform = ax.transAxes, fontsize = 10)
#save_lc_img_path='/Users/brettlv/Downloads/Mrk6/Mrk6_5100_Hbeta_lc.png'
#if not os.path.exists(save_lc_img_path):   
#plt.savefig(save_lc_img_path,dpi=400, transparent=False, bbox_inches='tight')

In [ ]:
lagdata_5100_Mrk6=get_lag_data(Mrk6,'mjd','F5100','e_F5100')
lagdata_FHb_Mrk6=get_lag_data(Mrk6,'mjd','FHb','e_FHb')

lagdata_5100_Mrk6

lagdata_FHb_Mrk6

## XRT V

In [ ]:
Mrk6_XRT=pd.read_csv('/Users/brettlv/blog/IHEP_sync/data/data_lag/Mrk6/data_XRT_Mrk6.csv')
Mrk6_V=pd.read_csv('/Users/brettlv/blog/IHEP_sync/data/data_lag/Mrk6/data_uvv_Mrk6.csv')

Mrk6_XRT.columns

Mrk6_V.columns

In [ ]:
name_test='Mrk 6'
D_cm_Mrk6=CLAGN_210919_mass[CLAGN_210919_mass['Name']=='Mrk 6']['D_cm'].values[0]
print(name_test,D_cm_Mrk6)
data_test_neo,data_test_mep=get_wise_data(name_test,datawise_dir='/Users/brettlv/blog/brettlv.github.io/pythoncode/changinglookAGN/CLAGN_wise_data/')
data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep)
data_rebinflux_neo,data_rebinflux_mep=get_wiseflux_rebin(data_rebin_neo,data_rebin_mep,D_cm_Mrk6)
datalag_w1_Mrk6,datalag_w2_Mrk6=get_wise_lc(data_rebinflux_neo,data_rebinflux_mep)

In [ ]:
Mrk6_XRT['logL']=np.log10(4*np.pi*D_cm_Mrk6**2)+Mrk6_XRT['logf']
Mrk6_XRT['logLerr']=Mrk6_XRT['logf_err']

Mrk6_V['logL']=np.log10(4*np.pi*D_cm_Mrk6**2)+Mrk6_V['logvf']
Mrk6_V['logLerr']=Mrk6_V['logvf_err']


timelabel,fluxlabel_w1,e_fluxlabel_w1=('mjd','logL', 'logLerr')
lag_Mrk6_V=get_lag_data(Mrk6_V,timelabel,fluxlabel_w1,e_fluxlabel_w1)
timelabel,fluxlabel_w1,e_fluxlabel_w1=('mjd','logL', 'logLerr')
lag_Mrk6_XRT=get_lag_data(Mrk6_XRT,timelabel,fluxlabel_w1,e_fluxlabel_w1)

datalag_w1

lag_Mrk6_XRT

lag_Mrk6_V

### lag_ W1 XRT V

In [ ]:
fig = plt.figure(figsize=(6,6))
fig.subplots_adjust(hspace=0.0, wspace = 0.0)
ax = fig.add_subplot(111)

ax.errorbar(datalag_w1_Mrk6['t'],datalag_w1_Mrk6['flux'],datalag_w1_Mrk6['err'],marker='.',ls='',label='W1')

ax.errorbar(lag_Mrk6_XRT['t'],lag_Mrk6_XRT['flux'],lag_Mrk6_XRT['err'],marker='.',ls='',label='XRT')

ax.errorbar(lag_Mrk6_V['t'],lag_Mrk6_V['flux'],lag_Mrk6_V['err'],marker='.',ls='',label='V')

set_ax_tick(ax)
ax.text(0.5, 0.9,name_test , horizontalalignment = 'center', verticalalignment = 'center',transform = ax.transAxes, fontsize = 10)

plot_secax(ax,mi_interval=365,ma_interval=365*2)
plt.xlabel('mjd')
plt.ylabel(r'$logL$')
plt.xlim(55000,59600)
plt.legend()
plt.show()
plt.close()

In [ ]:
fig = plt.figure(figsize=(6,6))
fig.subplots_adjust(hspace=0.0, wspace = 0.0)
ax = fig.add_subplot(111)

#ax.errorbar(datalag_w1_Mrk6['t'],datalag_w1_Mrk6['flux'],datalag_w1_Mrk6['err'],marker='.',ls='',label='W1')

ax.errorbar(lag_Mrk6_XRT['t'],lag_Mrk6_XRT['flux'],lag_Mrk6_XRT['err'],marker='.',ls='',label='XRT')
ax.errorbar(lag_Mrk6_V['t'],lag_Mrk6_V['flux'],lag_Mrk6_V['err'],marker='.',ls='',label='V')

set_ax_tick(ax)
ax.text(0.5, 0.9,name_test , horizontalalignment = 'center', verticalalignment = 'center',transform = ax.transAxes, fontsize = 10)

plot_secax(ax,mi_interval=365,ma_interval=365*2)
plt.xlabel('mjd')
plt.ylabel(r'$logL$')
plt.xlim(58500,59600)
plt.legend()
plt.show()
plt.close()

## run_ccf

In [ ]:
labelname1="5100A"
labelname2="Hbeta"
dataname1=lagdata_5100_Mrk6
dataname2=lagdata_FHb_Mrk6
sourcename='Mrk6'
rangemin,rangemax=57740,58000
lagrangel=-50
lagrangeh=100

In [ ]:
run_ccf(rootdir_ccf,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel,lagrangeh)

In [ ]:
plot_ccf(rootdir_ccf,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel=-10,lagrangeh=100)

## plot_ccf_injupyter

In [ ]:
plot_ccf_injupyter(rootdir_ccf,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel,lagrangeh)

## run_javelin

In [ ]:
labelname1="5100A"
labelname2="Hbeta"
dataname1=lagdata_5100_Mrk6
dataname2=lagdata_FHb_Mrk6
sourcename='Mrk6'

rangemin,rangemax=57740,58000
lagrangel=-50
lagrangeh=100
widthl=0
widthh=200

In [ ]:
run_javelin(rootdir_javelin,sourcename,labelname1,labelname2,
            dataname1,dataname2,rangemin,rangemax,lagrangel,lagrangeh,widthl,widthh)

# Mrk 1018

# NGC 1566

In [ ]:
#Mrk530_XRT=pd.read_csv('/Users/brettlv/blog/IHEP_sync/data/data_lag/Mrk530/data_XRT_Mrk530.csv')
NGC1566_V=pd.read_csv('/Users/brettlv/blog/IHEP_sync/data/data_lag/NGC1566/NGC1566_lag_combined_V.csv')

In [ ]:
name_test='NGC 1566'
D_cm_NGC1566=CLAGN_210919_mass[CLAGN_210919_mass['Name']=='NGC 1566']['D_cm'].values[0]
print(name_test,D_cm_NGC1566)
data_test_neo,data_test_mep=get_wise_data(name_test,datawise_dir='/Users/brettlv/blog/brettlv.github.io/pythoncode/changinglookAGN/CLAGN_wise_data/')
data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep)
data_rebinflux_neo,data_rebinflux_mep=get_wiseflux_rebin(data_rebin_neo,data_rebin_mep,D_cm_NGC1566)
datalag_w1_NGC1566,datalag_w2_NGC1566=get_wise_lc(data_rebinflux_neo,data_rebinflux_mep)

In [ ]:
data_rebinflux_mep

In [ ]:
wave6[0],freq6[0]

In [ ]:
NGC1566_V['logvf']=np.log10(NGC1566_V['flux'])-26+np.log10(freq6[0])
NGC1566_V['logvferr']=NGC1566_V['err']/NGC1566_V['flux']/np.log(10)
NGC1566_V['logL']=np.log10(4*np.pi*D_cm_NGC1566**2)+NGC1566_V['logvf']
NGC1566_V['logLerr']=NGC1566_V['logvferr']

timelabel,fluxlabel_w1,e_fluxlabel_w1=('t','logL', 'logLerr')
lag_NGC1566_V=get_lag_data(NGC1566_V,timelabel,fluxlabel_w1,e_fluxlabel_w1)

In [ ]:
timelabel,fluxlabel,e_fluxlabel='t','flux', 'err'
mean_mjds,mean_counts,mean_counts_e=median_lc(lag_NGC1566_V,timelabel,fluxlabel,e_fluxlabel,interval=30)
lag_NGC1566_V_rebin30=pd.DataFrame({'t':mean_mjds,'flux':mean_counts,\
                                  'err':mean_counts_e},\
                                  index=None,columns=['t','flux','err'])

mean_mjds,mean_counts,mean_counts_e=median_lc(lag_NGC1566_V,timelabel,fluxlabel,e_fluxlabel,interval=20)
lag_NGC1566_V_rebin20=pd.DataFrame({'t':mean_mjds,'flux':mean_counts,\
                                  'err':mean_counts_e},\
                                  index=None,columns=['t','flux','err'])

mean_mjds,mean_counts,mean_counts_e=median_lc(lag_NGC1566_V,timelabel,fluxlabel,e_fluxlabel,interval=10)
lag_NGC1566_V_rebin10=pd.DataFrame({'t':mean_mjds,'flux':mean_counts,\
                                  'err':mean_counts_e},\
                                  index=None,columns=['t','flux','err'])

In [ ]:
lag_NGC1566_V.columns

In [ ]:
fig = plt.figure(figsize=(6,6))
fig.subplots_adjust(hspace=0.0, wspace = 0.0)
ax = fig.add_subplot(111)

ax.errorbar(datalag_w1_NGC1566['t'],datalag_w1_NGC1566['flux'],datalag_w1_NGC1566['err']
            ,marker='.',ls='',label='W1')

ax.errorbar(lag_NGC1566_V_rebin10['t'],lag_NGC1566_V_rebin10['flux'],
            lag_NGC1566_V_rebin10['err']
            ,marker='.',ls='',label='V')

'''ax.errorbar(lag_NGC1566_V['t'],lag_NGC1566_V['flux'],
            lag_NGC1566_V['err']
            ,marker='.',ls='',label='V')'''



'''ax.errorbar(lag_NGC1566_V_rebin30['t'],lag_NGC1566_V_rebin30['flux'],
            lag_NGC1566_V_rebin30['err']
            ,marker='.',ls='',label='V')'''


'''ax.errorbar(lag_NGC1566_V_rebin20['t'],lag_NGC1566_V_rebin20['flux'],
            lag_NGC1566_V_rebin20['err']
            ,marker='.',ls='',label='V')
'''
#ax.errorbar(lag_Mrk6_XRT['t'],lag_Mrk6_XRT['flux'],lag_Mrk6_XRT['err'],marker='.',ls='',label='XRT')

#ax.errorbar(lag_NGC1566_V['t'],lag_NGC1566_V['flux'],lag_NGC1566_V['err'],marker='.',ls='',label='V')
ax.text(0.5, 0.9,name_test , horizontalalignment = 'center', verticalalignment = 'center',transform = ax.transAxes, fontsize = 10)

set_ax_tick(ax)
plot_secax(ax,mi_interval=365,ma_interval=365*2)
plt.xlabel('mjd')
plt.ylabel(r'$logL$')
plt.xlim(55000+200,59900)
plt.legend()


## run_ccf

In [ ]:
labelname1="V"
labelname2="W1"
dataname1=lag_NGC1566_V
dataname2=datalag_w1_NGC1566
sourcename='NGC1566'

rangemin,rangemax=57200,59500
lagrangel=0
lagrangeh=180


In [ ]:
run_ccf(rootdir_ccf,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel,lagrangeh)
plot_ccf(rootdir_ccf,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel,lagrangeh)
plot_ccf_injupyter(rootdir_ccf,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel,lagrangeh)

In [ ]:
lagrangel=0
lagrangeh=100
run_ccf(rootdir_ccf,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel,lagrangeh)
plot_ccf(rootdir_ccf,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel,lagrangeh)
plot_ccf_injupyter(rootdir_ccf,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel,lagrangeh)

## run_javelin

In [ ]:
labelname1="V"
labelname2="W1"
dataname1=lag_NGC1566_V
dataname2=datalag_w1_NGC1566
sourcename='NGC1566'

rangemin,rangemax=57200,59500
lagrangel=50
lagrangeh=100
widthl=0
widthh=100

In [ ]:
run_javelin(rootdir_javelin,sourcename,labelname1,labelname2,
            dataname1,dataname2,rangemin,rangemax,lagrangel,lagrangeh,widthl,widthh)

plot_javelin(rootdir_javelin,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel,lagrangeh)


phofchain = "%s/%s_mcmc_%s.dat"%(sourcename,labelname1,labelname2) 
javelin_lag_data=pd.read_csv(phofchain,header=None,sep=' ')
lag_javelin=javelin_lag_data[2]
width_javelin_data=javelin_lag_data[3]

perclim = 84.1344746
centau = stats.scoreatpercentile(lag_javelin, 50)
centau_uperr = (stats.scoreatpercentile(lag_javelin, perclim))-centau
centau_loerr = centau-(stats.scoreatpercentile(lag_javelin, (100.-perclim)))

width_javelin = stats.scoreatpercentile(width_javelin_data, 50)
width_javelin_uperr = (stats.scoreatpercentile(width_javelin_data, perclim))-width_javelin
width_javelin_loerr = width_javelin-(stats.scoreatpercentile(width_javelin_data, (100.-perclim)))

print(centau,width_javelin)

In [ ]:
plot_javelin_injupyter(rootdir_javelin,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel, lagrangeh)

# Mrk 530

In [ ]:
Mrk530_XRT=pd.read_csv('/Users/brettlv/blog/IHEP_sync/data/data_lag/Mrk530/data_XRT_Mrk530.csv')
Mrk530_V=pd.read_csv('/Users/brettlv/blog/IHEP_sync/data/data_lag/Mrk530/data_uvv_Mrk530.csv')

In [ ]:
name_test='Mrk 530'
D_cm_Mrk530=CLAGN_210919_mass[CLAGN_210919_mass['Name']=='Mrk 530']['D_cm'].values[0]
print(name_test,D_cm_Mrk530)
data_test_neo,data_test_mep=get_wise_data(name_test,datawise_dir='/Users/brettlv/blog/brettlv.github.io/pythoncode/changinglookAGN/CLAGN_wise_data/')
data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep)
data_rebinflux_neo,data_rebinflux_mep=get_wiseflux_rebin(data_rebin_neo,data_rebin_mep,D_cm_Mrk530)
datalag_w1_Mrk530,datalag_w2_Mrk530=get_wise_lc(data_rebinflux_neo,data_rebinflux_mep)

In [ ]:
Mrk530_XRT['logL']=np.log10(4*np.pi*D_cm_Mrk530**2)+Mrk530_XRT['logf']
Mrk530_XRT['logLerr']=Mrk530_XRT['logf_err']

Mrk530_V['logL']=np.log10(4*np.pi*D_cm_Mrk530**2)+Mrk530_V['logvf']
Mrk530_V['logLerr']=Mrk530_V['logvf_err']


timelabel,fluxlabel_w1,e_fluxlabel_w1=('mjd','logL', 'logLerr')
lag_Mrk530_V=get_lag_data(Mrk530_V,timelabel,fluxlabel_w1,e_fluxlabel_w1)
timelabel,fluxlabel_w1,e_fluxlabel_w1=('mjd','logL', 'logLerr')
lag_Mrk530_XRT=get_lag_data(Mrk530_XRT,timelabel,fluxlabel_w1,e_fluxlabel_w1)

In [ ]:
fig = plt.figure(figsize=(6,6))
fig.subplots_adjust(hspace=0.0, wspace = 0.0)
ax = fig.add_subplot(111)

ax.errorbar(datalag_w1_Mrk530['t'],datalag_w1_Mrk530['flux'],datalag_w1_Mrk530['err'],marker='.',ls='',label='W1')

ax.errorbar(lag_Mrk530_XRT['t'],lag_Mrk530_XRT['flux'],lag_Mrk530_XRT['err'],marker='.',ls='',label='XRT')

ax.errorbar(lag_Mrk530_V['t'],lag_Mrk530_V['flux'],lag_Mrk530_V['err'],marker='.',ls='',label='V')

set_ax_tick(ax)
plot_secax(ax,mi_interval=365,ma_interval=365*2)
plt.xlabel('mjd')
plt.ylabel(r'$logL$')
plt.xlim(55000,59600)
plt.legend()
ax.text(0.5, 0.9,name_test , horizontalalignment = 'center', verticalalignment = 'center',transform = ax.transAxes, fontsize = 10)


# Mrk926

## Mrk926_MAXI

In [ ]:
Mrk926_MAXI_process_2to6=pd.read_csv('%s/MAXIlc/Mrk_926_lc/Mrk_926_2.0-6.0keV_gsclc.dat'%CLAGN_statistic_plot_dir,header=None,delimiter='\s+')
Mrk926_MAXI_process_6to20=pd.read_csv('%s/MAXIlc/Mrk_926_lc/Mrk_926_6.0-20.0keV_gsclc.dat'%CLAGN_statistic_plot_dir,header=None,delimiter='\s+')
Mrk926_MAXI_process_2to6.columns=['t_start','t_end','rate','rate_err']
Mrk926_MAXI_process_6to20.columns=['t_start','t_end','rate','rate_err']

In [ ]:
lag_Mrk926_MAXI_process_2to6 =get_lag_data(Mrk926_MAXI_process_2to6,'t_start','rate','rate_err')
lag_Mrk926_MAXI_process_6to20=get_lag_data(Mrk926_MAXI_process_6to20,'t_start','rate','rate_err')

In [ ]:
Mrk926_MAXI_process_ratio=pd.read_csv('%s/MAXIlc/Mrk_926_lc/Mrk_926_ratio_gsclc.dat'%CLAGN_statistic_plot_dir,header=None,delimiter='\s+')
#Mrk926_MAXI_process_gsc=pd.read_csv('%s/MAXIlc/Mrk_926_lc/Mrk_926_gsc.flc'%CLAGN_statistic_plot_dir,header=None,delimiter='\s+')

Mrk926_MAXI_process_2to6

In [ ]:
Mrk926_MAXI=pd.read_csv('%s/MAXIlc/Mrk926_J2304-086_g_lc_1day_all.dat.txt'%CLAGN_statistic_plot_dir,header=None,delimiter='\s+')

Mrk926_MAXI.columns=['mjd','2-20keV[ph/s/cm2]', 'err_2-20keV', '2-4keV', 'err_2-4keV', '4-10keV', 'err_4-10keV' ,'10-20keV', 'err_10-20keV' ]

#Mrk926_MAXI

data=Mrk926_MAXI[Mrk926_MAXI['mjd']>57000]
data=drop_index(data)
timelabel,fluxlabel,e_fluxlabel='mjd','2-20keV[ph/s/cm2]', 'err_2-20keV'
mean_mjds,mean_counts,mean_counts_e=median_lc(data,timelabel,fluxlabel,e_fluxlabel,interval=180)
lag_Mrk926_Xray_rebin180=pd.DataFrame({'t':mean_mjds,'flux':mean_counts,\
                                  'err':mean_counts_e},\
                                  index=None,columns=['t','flux','err'])

mean_mjds,mean_counts,mean_counts_e=median_lc(data,timelabel,fluxlabel,e_fluxlabel,interval=90)
lag_Mrk926_Xray_rebin90=pd.DataFrame({'t':mean_mjds,'flux':mean_counts,\
                                  'err':mean_counts_e},\
                                  index=None,columns=['t','flux','err'])

mean_mjds,mean_counts,mean_counts_e=median_lc(data,timelabel,fluxlabel,e_fluxlabel,interval=60)
lag_Mrk926_Xray_rebin60=pd.DataFrame({'t':mean_mjds,'flux':mean_counts,\
                                  'err':mean_counts_e},\
                                  index=None,columns=['t','flux','err'])


mean_mjds,mean_counts,mean_counts_e=median_lc(data,timelabel,fluxlabel,e_fluxlabel,interval=30)
lag_Mrk926_Xray_rebin30=pd.DataFrame({'t':mean_mjds,'flux':mean_counts,\
                                  'err':mean_counts_e},\
                                  index=None,columns=['t','flux','err'])


In [ ]:
name_test='Mrk 926'
D_cm_Mrk926=CLAGN_210919_mass[CLAGN_210919_mass['Name']=='Mrk 926']['D_cm'].values[0]
print(name_test,D_cm_Mrk926)

In [ ]:
name_test='Mrk 926'
D_cm_Mrk926=CLAGN_210919_mass[CLAGN_210919_mass['Name']=='Mrk 926']['D_cm'].values[0]
print(name_test,D_cm_Mrk926)
data_test_neo,data_test_mep=get_wise_data(name_test,datawise_dir='/Users/brettlv/blog/brettlv.github.io/pythoncode/changinglookAGN/CLAGN_wise_data/')
data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep)
data_rebinflux_neo,data_rebinflux_mep=get_wiseflux_rebin(data_rebin_neo,data_rebin_mep,D_cm_Mrk926)
datalag_w1_Mrk926,datalag_w2_Mrk926=get_wise_lc(data_rebinflux_neo,data_rebinflux_mep)

## run_ccf

In [ ]:
labelname1="MAXI_bin30"
labelname2="W1"
dataname1=lag_Mrk926_Xray_rebin30
dataname2=datalag_w1_Mrk926
sourcename='Mrk926'

rangemin,rangemax=56000,59500
lagrangel=200
lagrangeh=400
run_ccf(rootdir_ccf,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel,lagrangeh)
plot_ccf(rootdir_ccf,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel,lagrangeh)
plot_ccf_injupyter(rootdir_ccf,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel,lagrangeh)

In [ ]:
labelname1="MAXI_bin60"
labelname2="W1"
dataname1=lag_Mrk926_Xray_rebin60
dataname2=datalag_w1_Mrk926
sourcename='Mrk926'

rangemin,rangemax=56000,59500
lagrangel=200
lagrangeh=400
run_ccf(rootdir_ccf,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel,lagrangeh)
plot_ccf(rootdir_ccf,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel,lagrangeh)
plot_ccf_injupyter(rootdir_ccf,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel,lagrangeh)

In [ ]:
labelname1="MAXI_6to20"
labelname2="W1"
dataname1=lag_Mrk926_MAXI_process_6to20
dataname2=datalag_w1_Mrk926
sourcename='Mrk926'

rangemin,rangemax=56000,59500
lagrangel=0
lagrangeh=400
run_ccf(rootdir_ccf,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel,lagrangeh)
plot_ccf(rootdir_ccf,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel,lagrangeh)
plot_ccf_injupyter(rootdir_ccf,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel,lagrangeh)

In [ ]:
labelname1="MAXI_2to6"
labelname2="W1"
dataname1=lag_Mrk926_MAXI_process_2to6
dataname2=datalag_w1_Mrk926
sourcename='Mrk926'

rangemin,rangemax=56000,59500
lagrangel=0
lagrangeh=400

run_ccf(rootdir_ccf,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel,lagrangeh)
plot_ccf(rootdir_ccf,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel,lagrangeh)
plot_ccf_injupyter(rootdir_ccf,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax,lagrangel,lagrangeh)

## run_javelin

In [ ]:
labelname1="MAXI_bin90"
labelname2="W1"
dataname1=lag_Mrk926_Xray_rebin90
dataname2=datalag_w1_Mrk926
sourcename='Mrk926'
rangemin,rangemax=56000,59500
lagrangel=100
lagrangeh=500
widthl=0
widthh=500

run_javelin(rootdir_javelin,sourcename,labelname1,labelname2,
            dataname1,dataname2,rangemin,rangemax,lagrangel,lagrangeh,widthl,widthh)

In [ ]:
fig = plt.figure(figsize=(4,4))
fig.subplots_adjust(hspace=0.15, wspace = 0.3)
ax = fig.add_subplot(1,1,1)
ax1=ax.twinx()
ax1.errorbar(datalag_w1_Mrk926['t'],datalag_w1_Mrk926['flux'],
            datalag_w1_Mrk926['err'],marker='.',ls='',label='W1')

ax.errorbar(lag_Mrk926_Xray_rebin90['t'],lag_Mrk926_Xray_rebin90['flux'],
            lag_Mrk926_Xray_rebin90['err'],marker='.',
                ls='',color='red')
#ax.set_xlim(55626+100,59900)
set_ax_tick(ax)
ax.set_xlabel('MJD (rebin 90)')
ax.set_ylabel('MAXI 2-20keV counts/s')

In [ ]:
fig = plt.figure(figsize=(4,4))
fig.subplots_adjust(hspace=0.15, wspace = 0.3)
ax = fig.add_subplot(1,1,1)
ax1=ax.twinx()
ax1.errorbar(datalag_w1_Mrk926['t'],datalag_w1_Mrk926['flux'],
            datalag_w1_Mrk926['err'],marker='.',ls='',label='W1')

ax.errorbar(lag_Mrk926_Xray_rebin180['t'],lag_Mrk926_Xray_rebin180['flux'],
            lag_Mrk926_Xray_rebin180['err'],marker='.',
                ls='',color='red')
#ax.set_xlim(55626+100,59900)
set_ax_tick(ax)
ax.set_xlabel('MJD (rebin 180)')
ax.set_ylabel('MAXI 2-20keV counts/s')

## Mrk926_BAT

In [ ]:
Mrk926_BAT=fits.open('/Users/brettlv/blog/IHEP_sync/data/data_lag/Mrk926_BAT_105m_crabweighted_1183.lc')

In [ ]:
Mrk926_BAT[1].data.columns

Mrk926_BAT[1].data['TIME']

In [ ]:
Mrk926_BAT_lc=fits.open('%s/BATlc/Mrk926.lc.fits'%CLAGN_statistic_plot_dir)

In [ ]:
min(Mrk926_BAT_lc[1].data['Time']),max(Mrk926_BAT_lc[1].data['Time'])

# Mrk 6

In [ ]:
name_test='Mrk 6'
D_cm_Mrk6=CLAGN_210919_mass[CLAGN_210919_mass['Name']=='Mrk 6']['D_cm'].values[0]
print(D_cm_Mrk6)

In [ ]:
name_test='NGC 1566'
D_cm_NGC1566=CLAGN_210919_mass[CLAGN_210919_mass['Name']=='NGC 1566']['D_cm'].values[0]
print(D_cm_NGC1566)